# MATH20014 Mathematical Programming: Group 3 Project Submission 

Group Members: Hamad Babur, Ryan Lam, Jiawei Li, Sherry Shen 


## Table of Contents

- [Introduction](#Intro)
- Chapter 1: [Introduction to `PyGame`](#PartA)
    - Section 1.1: [Animating Ball Motion](#S1)
    - Section 1.2: [Sierpenski Triangle](#S2)
    - Section 1.3: [Fractals and Trees](#S3)
- Chapter 2: [Applications of `Pygame`](#PartB)
    - Section 2.1: [Julia and Mandelbrot Sets](#S4)
        - Part 2.1.1: [Animation of Julia Sets](#S4.1)
        - Part 2.1.2: [Animation of Mandelbrot Sets](#S4.2)
    - Section 2.2: [Tower of Hanoi](#S5)
        - Part 2.2.1 [Definitions and Objectives](#S5.1)
        - Part 2.2.2 [Producing a Solution](#S5.2)
        - Part 2.2.3 [Identifying Valid Moves](#S5.3)
        - Part 2.2.4 [Animation of the Puzzles](#S5.4)
        - Part 2.2.5 [Extensions](#S5.5)
- [Conclusion](#Conc)
- [References and Bibilography](#Bib)

## Introduction <a name='Intro'></a>


Recursion is one of the most powerful tools in programming. It is not only a rather intuitive mathematical concept, but it is also easy to implement in any programming language. In this Python project, we shall both visually and mathematically appreciate the power of recursion.  

We shall focus on a set of modules collectively known as `PyGame` (see [(Shinners, 2021)](#PyGame_Intro)), as they provide a concise way for us to produce interactive animations within Python. The language of recursion naturally leads to the study of *fractals*, _id est_ self-repeating patterns and geometric objects. As we shall see later, a lot of patterns in nature are also closely related to fractals. 

The first chapter of the project consists of three short parts, aiming to understand the power of the `PyGame` module. Firstly, in section 1.1, we shall introduce and develop a simple PyGame animation function bouncing_ball, in which one or more balls bounce in a square box in two dimensions with various constraints. In section 1.2, we shall introduce fractals through the Sierpinski triangle and make an animation of how the Sierpinski triangles are produced. In section 1.3, we shall explore a particular construction and animation of a recursively defined “tree”. 

The second chapter of the project shall focus on some interesting applications when we shall integrate PyGame with different programming techniques. In section 2.1, we shall look at Julia sequences and we shall animate the Julia set as a transition from one picture to another. In addition to this, we shall look at Mandelbrot sequences, and then we shall create a program that zooms in and out of the Mandelbrot set. Finally, in section 2.2, we shall explore a classical puzzle - the Towers of Hanoi. We shall briefly look at the solution, the animation, and other space and time complexity issues.

In [1]:
# Required Imports:
import pygame, os, timeit
import matplotlib.pyplot as plt
import numpy as np
# Required Files: intro_ball.gif, intro_ball1.gif, intro_ball2.gif, segoeui.ttf

pygame 2.0.1 (SDL 2.0.14, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


<a id='PartA'></a>
## Chapter 1: Introduction to `PyGame`


In this part, we shall demonstrate animating and drawing in PyGame, which is very useful for our applications in [Chapter 2](#PartB).

<a id='S1'></a>
### Section 1.1: Animating Ball Motion 


In this section, we shall include some brief code to outline the main functionality of `PyGame`. For this, we shall recall that we are given a function `bouncing_ball` which allow users to play/pause an animation of a ball bouncing on a screen. We shall further develop it such that:

1.	The user should be able to change the original position of the ball and should be able to change the speed of the ball.

2.	The ball should slow down under the effect of gravity.

3.	Two or more balls bounce within the same square (and of one another).

We shall call the developed function `bouncing_ball_ext`.

To do this, we can utilize the power of classes in Python. We can observe that intuitively, a moving ball should have a size, its current position and its current velocity, and we shall introduce ourselves to the following definition:


In [2]:
class Ball(object):
    '''
    The class Ball encapsulate all the information needed to 
    draw a ball on the PyGame module.
    '''
    def __init__(self, x, y, x_step, y_step, size=50):
        '''
        Initialization of the class Ball. 
        (x, y) - the coordinate relative to the screen, 
        both of them are float from 0 to 1.
        (x_step, y_step) - the velocity of the ball (pixel per frame).
        size - optional argument, size of the ball.
        '''
        self.x = x
        self.y = y
        self.x_step = x_step
        self.y_step = y_step
        self.size = size

Note that we have set the attribute `size` as optional, as this would simplify the initialization of the object a bit.


<a id = "anim-explain"></a>
Now we shall discuss as to how we can implement the animation in the `PyGame` module. This will be a generic design that will be applied to every animation we shall create in the later sections.


> **Note:** In `pygame`:
> 
> 1. A click of a mouse button or pressing of a keyboard button consists of the following two actions.
>
>   - Pressing down on a button, which is detected by `event.type == pygame.KEYDOWN` and
>
>   - Releasing a button, which we will not being using.
>
> Note that `PyGame` considers the above as two different events.
>
> 2. A co-ordinate grid is implemented. And the $y$-axis $y=0$ is at the top of the screen, that is why when gravity is negative, the ball slows down at the top.

We only use the pressing action to describe a press of the keyboard in this particular function. This is because, for this function, the only time the keyboard is used is when a button/key is being pressed.





Firstly, an animation program needs to load the files, which can be done by using commnads like `pygame.bilt`, or `pygame.image.load`. Then, we need to initialise `PyGame` with an appropriate screen size and set the time `clock`. Then, we need to load the first picture, and wait for the clocks ticks for a few milliseconds, and then load the next picture. This goes on until the last picture. 

From here we add in extra features like pausing and speeding up and down the animation. The steps below describe on what the program should do in detail.

> 1. Configure screen size and colour as tuples, and store the clock time in a variable.
>
> 2. Initialise `PyGame`, and set the program window using the screen size tuple used earlier and the title.
>
> 3. Load the picture above a white background and strech it to the program window.
>
> 4. Use the current speed factor and the clock time to wait for a specified amount of time, and then load the next picture, replacing the current picture.
>
> 5. Repeat Step 4 until the last image loads, then go back to the first image and go back to Step 5.

In addition to this, the program needs to allow the user to stop the animation, speed up the animation and slow down the animation. In order to do this, we shall detect keyboard presses from the user:

> - If the user initiates the exit sequence, stop running the program and quit the PyGame window.
>
> - If the user presses the `Space` button, load the current picture rather than loading the next picture in Step 5 to create a static image.
>
> - If the user presses the `Up Arrow` button, increase the speed factor to speed up Step 4 above and as a result, this speeds up the animation.
>
> - If the user presses the `Down Arrow` button, decrease the speed factor to slow down Step 4 above and as a result, this slows down the animation.

Now we present the implementation of `bouncing_ball_ext` in Python.

In [3]:
def bouncing_ball_ext(ball, speed_factor=5,gravity=9.81):
    '''
    Play a bouncing ball animation according to the inputs:
    Inputs:
    ball - an object from the class Ball
    speed_factor - the frames per second of animation
    gravity - the drag factor affecting the motion of the ball (positive is upwards)
    (N.B., gravity = 0 corresponds to no gravity, obviously)
    Output: None
    '''
    # Required variables, x0, y0 is the starting coordinate of the ball
    screen_size = (screen_width, screen_height) = (1080, 800)
    white = (255,255,255)
    x0, y0 = (screen_width - ball.size)*ball.x, (screen_height - ball.size)*ball.y
    g = -gravity
    # Used for the pause time in the animation while loop below
    frames_per_second = 10 + 10*speed_factor
    clock = pygame.time.Clock()
    # Set up the animation     
    pygame.init()
    screen = pygame.display.set_mode(screen_size)
    # Titles and Instructions
    caption = f'Bouncing Ball with Two Balls with gravity factor {-gravity},'
    caption += '                   '
    caption += '(Keystroke:  \'Space\' to start or pause, \'ArrowUp/ArrowDown\' to speed up/down)'
    caption += '                   '
    pygame.display.set_caption(f'{caption} speed = {frames_per_second}')
    # Load the image
    ball_fig = pygame.image.load("intro_ball.gif")
    ball_fig = pygame.transform.scale(ball_fig, (ball.size, ball.size))
    # Assign an pygame.Rect object to the ball
    ball_rect = pygame.Rect(x0,y0,ball.size,ball.size)
    # Initalize the Screen
    screen.fill(white)
    screen.blit(ball_fig, ball_rect)
    pygame.display.flip()
    # Variable to detect change
    keep_running = True
    move_ball = False
    # Animation loop 
    while keep_running:
        # If a keyboard event happens register it... 
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                keep_running = False
            # Start/Stop Animation
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                move_ball = not move_ball
            # Speed change if ArrowUp/ArrowDown is pressed
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                frames_per_second = (4/3)* frames_per_second
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                frames_per_second = 0.75 * frames_per_second
            # Re-initiate caption to display change
            pygame.display.set_caption(f'{caption} speed = {frames_per_second}')
        if move_ball:
            # Move the ball a step under gravity
            ball.y_step += g/(frames_per_second)
            ball_rect.x += ball.x_step
            ball_rect.y += ball.y_step
            if ball_rect.left < 0 or ball_rect.right > screen_width:
                ball.x_step = - ball.x_step
            if ball_rect.top < 0 or ball_rect.bottom > screen_height:
                ball.y_step = - ball.y_step
        # Redraw and update the screen
        screen.fill(white)
        screen.blit(ball_fig, ball_rect)
        pygame.display.flip()
        clock.tick(frames_per_second)
    pygame.quit()
    return None 

We can test our function using one instance of our class `Ball`.

In [4]:
# A ball positioned at the middle, with velocity (1.2, 3).
ballA = Ball(0.5, 0.5, 1.2, 3)
bouncing_ball_ext(ballA)

Next, we can implement collision between two balls. 

We assume every collision is *elastic*, that is,  we only have to exchange the velocity between the two balls when a collision is detected.

In [5]:
def bouncing_ball_two(ball1, ball2, speed_factor=5,gravity=9.81):
    '''
    Play an animation similar to bouncing_ball with two balls with elastic collision between the balls.
    The input (and code, actually) are similar with an extra parameter ball2 in the Class Ball.
    '''
    # Required variables, x0, y0 is the starting coordinate of the ball
    screen_size = (screen_width, screen_height) = (1080, 800)
    white = (255,255,255)
    x0, y0 = (screen_width - ball1.size)*ball1.x, (screen_height - ball1.size)*ball1.y
    x1, y1 = (screen_width - ball1.size)*ball2.x, (screen_height - ball2.size)*ball2.y
    g = -gravity
    # Used for the pause time in the animation while loop below
    frames_per_second = 10 + 10*speed_factor
    clock = pygame.time.Clock()
    # Set up the animation     
    pygame.init()
    screen = pygame.display.set_mode(screen_size)
    # Titles and Instructions
    caption = f'Bouncing Ball with Two Balls with gravity factor {-gravity}'
    caption += '                              '
    caption += '(Keystroke:  \'Space\' to start or pause)'
    pygame.display.set_caption(f'{caption} speed = {frames_per_second}')
    # Load the image and rescale
    ball1_fig = pygame.image.load("intro_ball1.gif")
    ball1_fig = pygame.transform.scale(ball1_fig, (ball1.size, ball1.size))
    ball1_rect = pygame.Rect(x0,y0,ball1.size,ball1.size)
    ball2_fig = pygame.image.load("intro_ball2.gif")
    ball2_fig = pygame.transform.scale(ball2_fig, (ball2.size, ball2.size))
    ball2_rect = pygame.Rect(x1,y1,ball2.size,ball2.size)
    # Initalize the Screen
    screen.fill(white)
    screen.blit(ball1_fig, ball1_rect)
    screen.blit(ball2_fig, ball2_rect)
    pygame.display.flip()
    # Variable to detect change
    keep_running = True
    move_ball = False
    # Animation loop 
    while keep_running:
        # Keyboard Event: Quit Button and Space to Start/Stop
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                keep_running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                move_ball = not move_ball
            # Speed change if ArrowUp/ArrowDown is pressed
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                frames_per_second = (4/3)* frames_per_second
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                frames_per_second = 0.75 * frames_per_second
            pygame.display.set_caption(f'{caption} speed = {frames_per_second}')
        if move_ball:
            ball1.y_step += g/(frames_per_second)
            ball2.y_step += g/(frames_per_second)
            # Move the ball a step and Screen Border Detection
            ball1_rect.x += ball1.x_step
            ball1_rect.y += ball1.y_step
            if ball1_rect.left < 0 or ball1_rect.right > screen_width:
                ball1.x_step = - ball1.x_step
            if ball1_rect.top < 0 or ball1_rect.bottom > screen_height:
                ball1.y_step = - ball1.y_step
            ball2_rect.x += ball2.x_step
            ball2_rect.y += ball2.y_step
            if ball2_rect.left < 0 or ball2_rect.right > screen_width:
                ball2.x_step = - ball2.x_step
            if ball2_rect.top < 0 or ball2_rect.bottom > screen_height:
                ball2.y_step = - ball2.y_step
            # Collision Detection 
            collide = ball1_rect.colliderect(ball2_rect)
            if collide:
                # If they collide, the velocity of the two balls exchange
                copy_ball1 = ball1
                ball1 = Ball(ball2.x, ball2.y, ball2.x_step, ball2.y_step)
                ball2 = Ball(copy_ball1.x, copy_ball1.y, copy_ball1.x_step , copy_ball1.y_step) 
        # Redraw and reinitiate              
        screen.fill(white)
        screen.blit(ball1_fig, ball1_rect)
        screen.blit(ball2_fig, ball2_rect)
        pygame.display.flip()
        clock.tick(frames_per_second)
    pygame.quit()
    return None 

Now we can play around with different balls in the program.

In [6]:
# Testing Cell
ballA = Ball(0.5, 0.5, 1.2, 3, 70)
ballB = Ball(0.5, 0.2, 1, -5, 100)
bouncing_ball_two(ballA, ballB, gravity =0, speed_factor =20)

We can also invite users to input their own parameters, this is very useful for testing different functions and will be used throughout the document. 

But first, we need a systemized way to ensure the user's input make sense.


In [7]:
def integer_input_validation(input_message, default_value, lower, upper):
    '''
    Prompt an integer input with input_message shown to user, 
    and require the input to be bounded between lower and upper.
    If the requirements are satisfied, return the integer user inputs. 
    Otherwise, return default_value.
    '''
    test_input = input(f'Please enter {input_message}. \
                       This is an integer between {lower} and {upper} inclusive.')
    try:
        test_int = int(test_input)
        if test_int <= upper and test_int >= lower:
            print(f'{input_message} set as the value {test_int}')
            return test_int
        else:
            print(f'Out of bounds: using the default value of {default_value}.')
            return default_value
    except:
        print(f'Type Error: using the default value of {default_value}.')
        return default_value
def float_input_validation(input_message, default_value, lower, upper):
    '''
    Prompt an float input with input_message shown to user, 
    and require the input to be bounded between lower and upper.
    If the requirements are satisfied, return the integer user inputs. 
    Otherwise, return default_value.
    '''
    test_input = input(f'Please enter {input_message}. \
                       This is a real number (float) between {lower} and {upper} inclusive.')
    try:
        # Try to Convert the message to a float
        test_int = float(test_input)
        if test_int <= upper and test_int >= lower:
            print(f'{input_message} set as the value {test_int}')
            return test_int
        else:
            print(f'Out of bounds: using the default value of {default_value}.')
            return default_value
    except:
        print(f'Type Error: using the default value of {default_value}.')
        return default_value

Now we are ready to provide an interactive version of the code.

In [8]:
def run_bouncing_ball():
    '''
    Interactively let the user choose which version of the animation is to be used.
    '''
    # Setting defaults and limits
    min_pos, max_pos = 0, 1
    min_vel, max_vel = -20, 20
    default_speed = 5
    default_r1 = 5
    default_theta1 = -2
    default_r2 = -8
    default_theta2 = 5
    default_position1 = 0.5
    default_position2 = 0.2
    default_gravity = 9.81
    # Get the parameters from the user. 
    x_pos = float_input_validation('initial x position', default_position1, min_pos, max_pos)
    y_pos = float_input_validation('initial y position', default_position2, min_pos, max_pos)
    x_step = float_input_validation('initial x velocity', default_r1, min_vel, max_vel)
    y_step = float_input_validation('initial y velocity', default_theta1, min_vel, max_vel)
    speed = float_input_validation('speed factor', default_speed, 0, 10)
    gravity = float_input_validation("gravity (if you do not want gravity, enter 0)", default_gravity,0, 20)
    test_ball = Ball(x_pos, y_pos,x_step, y_step)
    # Prompt the user whether an extra ball is needed
    extra_ball = input("Do you want an extra ball? Enter 'y' if you need one, 'n' otherwise. ")
    if extra_ball == 'y':
        # Parameters of the extra ball, again from the user.
        print('Please provide information about the extra ball...')
        x_pos2 = float_input_validation('initial x position', default_position2, min_pos, max_pos)
        y_pos2 = float_input_validation('initial y position', default_position1, min_pos, max_pos)
        x_step2 = float_input_validation('initial x velocity', default_r2, min_vel, max_vel)
        y_step2 = float_input_validation('initial y velocity', default_theta2, min_vel, max_vel)
        test_ball2 = Ball(x_pos2, y_pos2,x_step2, y_step2)
        bouncing_ball_two(test_ball, test_ball2, speed, gravity)
    elif extra_ball == 'n':
        print('Running with 1 ball!')
        bouncing_ball_ext(test_ball,speed, gravity)
    else:
        print('Invalid input! Running with 1 ball!')
        bouncing_ball_ext(test_ball,speed, gravity)
    return None

In [9]:
# Test Cell for interactive input
run_bouncing_ball()

Type Error: using the default value of 0.5.
Type Error: using the default value of 0.2.
Type Error: using the default value of 5.
Type Error: using the default value of -2.
Type Error: using the default value of 5.
Type Error: using the default value of 9.81.
Invalid input! Running with 1 ball!



### Section 1.2: Sierpenski Triangle 
<a id='S2'></a>

Animating fractal constructions gives us a direct insight into how the function generating the fractal operates recursively. A typical example is the *Sierpinski Triangle*. It is a self-similar fractal, for which can be created by starting with one large, equilateral triangle, and then repeatedly cutting smaller triangles out of its centre. Interested readers are referred to [(Falconer, 1990)](#Falconer1990) for details. 

We are given a simple `PyGame` animation `draw_sierpinski`. Note that this function depends on `make_sierpinski`, which calls on itself using `depth-1` recursively. Similar constructs shall appear in our [next section](#S3) while drawing a recursive tree as well.

In [10]:
def make_sierpinski(depth, triangle, triangle_list):
    '''
    Function inputs: 
    - depth (of recursion), 
    - triangle (vertex coordinates)
    - triangle_list (list of triange coordinates)
    Modifies triangle_list: all the depth 1 (bottom) triangles are added 
    to this list (using recursion relative to the input triangle)
    '''
    (x0,y0) = triangle[0]
    (x1,y1) = triangle[1]
    (x2,y2) = triangle[2]
    # Maximum depth reached (going down) so add this triangle to the list
    if depth == 1:
        triangle_list.append(triangle)
        return None 
    # Otherwise split triangle into three sub triangles
    midpoint_A = (x0 + (x1-x0)/2.0, y0)
    midpoint_B = (x0 + (x2-x0)/2.0, y2 + (y0-y2)/2.0)
    midpoint_C = (x2 + (x1-x2)/2.0, y2 + (y1-y2)/2.0)
    # Recursive call on triangles
    new_triangle = ((x0,y0), midpoint_A, midpoint_B)
    make_sierpinski(depth-1, new_triangle, triangle_list)
    new_triangle = (midpoint_A, (x1,y1), midpoint_C)
    make_sierpinski(depth-1,new_triangle,triangle_list)
    new_triangle = (midpoint_B, midpoint_C, (x2,y2))
    make_sierpinski(depth-1, new_triangle, triangle_list)    
    return None

We shall improve and develop on this basis to meet the following three conditions:

1.	The user can be able to choose the depth of the triangle and stop and start the animation.

2.	The user is also able to change the speed of the animation.

3.	Adding colours (in RGB value) to the triangles and the background.

We shall call our developed function `draw_sierpinski_ext`. 

In [11]:
def draw_sierpinski_ext(depth=6, speed = 50,colour=(0, 0, 255),backgroundColour = (255,255,255)):
    '''
    Function that draws the Sierpinski triangle as an animation. 
    Inputs: 
    depth - Depth of the Triangle as specified in make_sierpinski
    speed - frame per second in the animation, can be adjusted using up and down arrows
    colour - Color of the triangle: An 3-integer tuple storing the RGB Values
    backgroundColour - Color of the background: An 3-integer tuple storing the RGB Values
    Outputs: None
    '''
    dimensions = (screen_width,screen_height) = (900, 862)
    master_triangle = ((50,800),(850,800),(450,62))
    clock = pygame.time.Clock()
    frames_per_second = speed
    # Make a list of all the triangle vertex coordinates of the given 
    # depth (in make_sierpinski we process  depth to work down to 1)
    triangle_list = []
    make_sierpinski(depth,master_triangle,triangle_list)
    # Initialise pygame, the screen display and title
    pygame.init()
    screen = pygame.display.set_mode(dimensions)
    caption = 'Sierpinski Triangle            '
    caption += '(1)  \'Space\' to start or pause    '
    caption += '(2)  \'Up/Down\' to change the speed'
    pygame.display.set_caption(f'{caption} (3) speed = {speed}')
    screen.fill(backgroundColour)
    pygame.display.flip()
    # Total number of triangles to be drawn 
    number_of_triangles = len(triangle_list)
    index = 0
    draw_triangle = False
    keep_running = True
    # Animation loop 
    while keep_running:
        for event in pygame.event.get():
            # Exit (at end of this iteration) using quit (e.g Ctrl-q or red button)
            if event.type == pygame.QUIT:
                keep_running = False
            if speed < 1:
                speed = 1
            # Start and pause the animation with the space key 
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                draw_triangle  = not draw_triangle
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                speed = (4/3)* speed
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                speed = 0.75 * speed
            pygame.display.set_caption(f'{caption} (3) speed = {speed}')
            frames_per_second = speed 
        # Keep draw next triangle with index 'index' if not told to pause and not complete
        if draw_triangle and index  < number_of_triangles:
            pygame.draw.polygon(screen,colour,triangle_list[index],0)
            pygame.display.update()
            clock.tick(frames_per_second)
            # Index uptate: index walks through triangle_list indices
            index += 1
    pygame.quit()
    return None

In [12]:
# Test Cell:
draw_sierpinski_ext()
# With more parameters
draw_sierpinski_ext(depth=8, speed = 60,colour=(255, 255, 255),backgroundColour = (171 ,31, 45))

An interactive version for testing is given below:

In [13]:
def run_sierpinski_ext(): 
    """
    Draw the sierpinski triangle using user-defined inputs. 
    """
    # Get required information from the user 
    depth = integer_input_validation('integer depth',6,1,10)
    speed = integer_input_validation('triangles to generate per second',50,1, 60)
    trig_R = integer_input_validation('R value for triangle Color',0,0,255)
    trig_G = integer_input_validation('G value for triangle Color',255,0,255)
    trig_B = integer_input_validation('B value for triangle Color',0,0,255)
    bg_R = integer_input_validation('R value for triangle Color',255,0,255)
    bg_G = integer_input_validation('G value for triangle Color',255,0,255)
    bg_B = integer_input_validation('B value for triangle Color',255,0,255)
    trig_color = (trig_R, trig_G, trig_B)
    bg_color = (bg_R, bg_G, bg_B)
    # Now run the animation with user-provided input
    draw_sierpinski_ext(depth, speed, trig_color, bg_color) 
    return None


In [14]:
# Test cell for interactive input
run_sierpinski_ext()

Type Error: using the default value of 6.
Type Error: using the default value of 50.
Type Error: using the default value of 0.
Type Error: using the default value of 255.
Type Error: using the default value of 0.
Type Error: using the default value of 255.
Type Error: using the default value of 255.
Type Error: using the default value of 255.


<a id='S3'></a>
### Section 1.3: Fractals and Trees 

In this section, we shall visualize constructing a fractal tree as outlined in Chapter 16 of [(Mandelbrot, 1999)](#M1999). 

We shall first outline how we are going to construct a tree for any non-negative integer depth. 

We start with a straight line called a **trunk at depth 0**.  The trunk splits into three lines, which we call **branches**. The left branch and right branch make an angle $\theta$ with the central branch, which is constructed by extending the trunk at depth 0. Then, all three branches can be considered as a **trunk of depth 1**, and each of them can split into three branches (i.e. trunk at depth 2) and so on. 

> **Definition**: the **tree of depth $n$ with angle $\theta$** as the collection of line of trunk at depth $0, 1, 2 \cdots n$ as outlined above. In particular, we will call the trunk at depth $n$ **leaves**.


Note that as we employ a recursive definition of the tree, the tree should have $3^n$ trunks.


Then, we shall use `PyGame` to create an animation of drawing the recursive tree. Meanwhile, four conditions need to be satisfied, and this is of the following:

1.	Users should be able to change the depth of the tree. 

2.	Users should be able to start and stop the drawing of the tree. 

3.	The leaves should be coloured green and the trunk should be coloured brown.

4.	Users should be able to change the speed of drawing the tree.


In [15]:
def tree_anim(depth, fps = 10, angle = np.pi/6, scale_factor = 0.8, init_len = 150):
    '''
    Plays the animation of drawing a recursive tree as outlined above.
    Inputs: 
    depth - The level of the tree as explained in the report.
    fps - Numbers of lines shown per second.
    angle - the angle theta mentioned in the definition.
    scale_factor - the length ratio between two trunk that are 1 depth away.
    init_let - the length of the trunk at depth 0.
    '''
    size = width, height = 1080, 800  
    brown = (150, 75, 0)
    green = (0, 255, 0)
    white = (255, 255, 255)  
    ans_list = []
    frames_per_second = fps
    clock = pygame.time.Clock()
    def branch(length, curr_pos, direction, depth_counter=0):
        '''
        A subfunction in tree_anim which alters the ans_list when called.
        Inputs:
        length - length of the trunk
        curr_pos - The starting point of the trunk (coordinate)
        direction - current facing angle of the tree
        depth_counter - current depth of the trunk being plotted
        '''
        if depth_counter > depth:
            return None
        else:
            angle_x = length * np.cos(direction)
            angle_y = length * np.sin(direction)
            next_position = (curr_pos[0] + angle_x, curr_pos[1] + angle_y)
            if depth_counter == depth:
                ans_list.append((green, curr_pos, next_position))
            else:
                ans_list.append((brown, curr_pos, next_position))
            new = length * scale_factor
            branch(new, next_position, direction-angle, depth_counter+1)
            branch(new, next_position, direction, depth_counter+1)
            branch(new, next_position, direction+angle, depth_counter+1)
    branch(init_len, (width//2, height), -np.pi/2)
    # Animation Initializations
    pygame.init()
    win = pygame.display.set_mode(size)
    pygame.display.set_caption(f"Fractal Tree of depth {depth} - Current Speed: {frames_per_second}")
    win.fill(white)
    # Key variables to keep track of
    keep_running = True
    move_ball = False
    index = 0
    # Animation loop 
    while keep_running:
        # Keyboard events
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                keep_running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                move_ball = not move_ball
            # Speed change if ArrowUp/ArrowDown is pressed
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                frames_per_second = (4/3)* frames_per_second
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                frames_per_second = 0.75 * frames_per_second
            pygame.display.set_caption(f"Fractal Tree of depth {depth} - Current Speed: {frames_per_second}")
        # Draw a line according to ans_list
        if move_ball and index < len(ans_list):
            col, pos, next_pos = ans_list[index]
            pygame.draw.line(win, col, pos, next_pos)
            index += 1
        # Redraw step
        pygame.display.flip()
        clock.tick(frames_per_second)
    pygame.quit()

In [16]:
# TEST CELL
tree_anim(5, fps=30)

An interactive version can be found below:


In [17]:
def run_tree(): 
    """
    Draw the recursive tree using user-defined inputs. 
    """
    # Get required information from the user 
    depth = integer_input_validation('integer depth',6,1,10)
    speed = integer_input_validation('branches to generate per second',50,1, 60)
    angle = float_input_validation('Angle between Branches',np.pi/6,0, np.pi)
    scale_factor = float_input_validation('Scale Factor',0.8,0, 1)
    init_len = integer_input_validation('Length of Branch at Depth 0',150,50, 200)
    # Now run the animation with user-provided input
    tree_anim(depth, speed, angle, scale_factor, init_len)
    return None

In [18]:
run_tree()

Type Error: using the default value of 6.
Type Error: using the default value of 50.
Type Error: using the default value of 0.5235987755982988.
Type Error: using the default value of 0.8.
Type Error: using the default value of 150.


<a id='PartB'></a>
## Chapter 2: Applications of `PyGame` 

In this part, we shall focus on two major applications of `PyGame`, namely exploring complex function dynamics in section 4, and a famous recreational puzzle in section 5.

<a id='S4'></a>
### Section 2.1: Julia and Mandelbrot Sets 

In light with the power of modern computers, dynamical systems become one of the most active research areas in mathematics. In particular, Mandelbrot in [(Mandelbrot, 1980)](#M1980) is interested in the dynamics of the complex function $f : \mathbb{C} \to \mathbb{C},  z \to \lambda z (1-z)$. 

By dynamics, we are interested in the following question:

> Given a value $c$, how do $f(c), f(f(c)), f(f(f(c))), ... f^n(c)$ behave? What is its value when $n$ tends to infinity?

Note that $f$ above can be transformed into $w \to w^2+D$ for some $D \in \mathbb{C}$ by doing a suitable translation $w \to z-a$. Hence, we are motivated to the following definition:

> **Definition:** Given $j_p \in \mathbb{C}$. The **filled in Julia set with parameter $\mathit{j_p}$** is defined as the values of $c$ for which the following recurrence relation such that the sequence $(z_n)_{n \geq 0}$ is *bounded*:
>
> $$ z_0 = c  \quad \text{and} \quad z_n = z_{n-1}^2 + j_p, \quad \text{for } n = 1,2,3,\dots $$
>
> We say that $(z_n)_{n \geq 0}$ is the **Julia sequence with parameter $j_p$ induced by $c$**. From this, we can see that $(z_n)_{n \geq 0}$ only relies on $c$ and the fixed parameter $j_p$.
>
> The boundary of the filled in Julia set is called a **Julia Set (with parameter $\mathit{j_p}$)**.

> **Definition:** The **Mandelbrot set $M$** is defined as the values of $c$ for which the following recurrence relation is bounded for all $(z_n)_{n \geq 0}$:
>
> $$ z_0 = 0  \quad \text{and} \quad z_n = z_{n-1}^2 + c, \quad \text{for } n = 1,2,3,\ldots$$
>
> We say that $(z_n)_{n \geq 0}$ is the **Mandelbrot sequence induced by $c$**. From this, we can see that $(z_n)_{n \geq 0}$ only relies on $c$.

We can immediately observe the similarity of the two definitions. In particular, we will take it as a fact without proof that, if $c \in M$, then the Julia set with parameter $c$ will be connected. In particular, the most interesting Julia set is when we take the parameter at the  boundary of the Mandelbrot set.

Mandelbrot set become one of the most well known figure in mathematics due to the fractal nature of the set. We will develop a program that allow user to zoom in to the figure to explore the self-repeating pattern.

<a id='S4.1'></a> 
#### Part 2.1.1 Animation of Julia Sets


Firstly, we shall look at Julia sets and animate images of Julia sets as transistioning slides.

We shall now create a function `julia` that calculates the largest term `n_max` for which each term $z_n$ of the corresponding Julia sequence with parameter $j_p$ induced by $c$ is bounded by $2$, otherwise the sequence $(z_n)_{n \geq 0}$ diverges as $n$ tends to infinity. To avoid long processing times, we shall set a maximum number of terms `max_n` that is looped through in the function and if it is reached, the function returns the value of `max_n`.

In [19]:
def julia(j_p, c, max_n):
    '''
    Calculates the largest number n_max for which each 
    term z_n of the corresponding Julia sequence
    with parameter j_p induced by c is bounded by 2. 
    
    Inputs: j_p, c - as in the definition of the julia set.
    max_n - Maximum Number of iterationns
    '''
    z = c
    n_max = 0
    # Iterate until term is not bounded by 2.
    while abs(z) <= 2:
        z = (z ** 2) + j_p
        n_max += 1
        # Break the loop early when the largest number n_max reaches 
        # the maximum number of iterations max_n.
        if n_max == max_n:
            break
    return n_max

We shall now create data for our Julia set. In order to do this, we need to create vectors `x_vector` and `y_vector` in order to create grid matrices `x_matrix` and `y_matrix` as input matricies. The vectors must be a number `c_intervals` of equally spaced intervals between the values `c_min` and `c_max`. 



We shall create the following function that takes a number `c_intervals` of equally spaced intervals between the values `c_min` and `c_max` and calculates `x_matrix` and `y_matrix`, and inputs the matrices into the `julia` function to create an output matrix `z_matrix`. In order to generate data needed for the set, we need `x_matrix`, `y_matrix` and `z_matrix`.

In [20]:
def julia_data(j_p, max_n, c_min, c_max, c_intervals):
    '''
    Calculates the matrices required for graphing the julia function.

    Input:
    j_p, max_n - as with the function julia
    c_min, c_max, c_intervals - used to creative an array of c to consider 
    whether they belong in the Filled in Julia set
    '''
    # Calculate the vectors required for the two matrices required for input.
    x_vector = np.linspace(c_min, c_max, c_intervals)
    y_vector = np.linspace(c_min, c_max, c_intervals)
    x_matrix, y_matrix = np.meshgrid(x_vector, y_vector)
    # Create a zero output matrix with the same dimensions as the input matrices.
    z_rows, z_cols = np.shape(x_matrix)
    z_matrix = np.zeros((z_rows, z_cols), dtype = int)
    # Calculate every element of the output matrix for every 
    # element in the input matrix using the julia function.
    for i in range(c_intervals):
        for j in range(c_intervals):
            c = x_matrix[i, j] + y_matrix[i, j] * 1j
            z_matrix[i, j] = julia(j_p, c, max_n)
    return x_matrix, y_matrix, z_matrix

We shall create a folder, so that we can export the images into an appropriate directory entitled "JULIA_GRAPHS".

In [21]:
# Obtain the current working directory.
currwd1 = str(os.getcwd())
# Create a subdirectory within the current working directory using the system terminal.
createdir1 = "mkdir \"{}//JULIA_GRAPHS\"".format(currwd1)
os.system(createdir1)

1

We shall now create a function that generates and exports the Julia set.

In [22]:
def julia_graphpic(j_p, max_n, c_min, c_max, c_intervals, 
plt_width, plt_height, filename):
    '''
    Exports a Julia graph as a picture with default resuloutions.
    Inputs: 
    j_p, max_n, c_min, c_max, c_intervals - as with julia_data.
    plt_width, plt_height - Default resolution of the figure (integers)
    filename - a string used to be the filename of the exported png file
    '''
    # Calculate the two input matrices and one output matrix using the julia_data function.
    x_matrix, y_matrix, z_matrix = julia_data(j_p, max_n, c_min, c_max, c_intervals)
    # Produce a Julia graph from the given matrices without the axes.
    plt.figure(figsize = (plt_width, plt_height))
    plt.pcolor(x_matrix, y_matrix, z_matrix, cmap = 'plasma', vmin = 0, vmax = max_n, shading = "auto")
    plt.axis("off")
    # Create a filepath directory string and export  
    # the aforementioned Julia graph into the filepath directory.
    currwd = os.getcwd()
    filepath = str(currwd) + "//JULIA_GRAPHS//{}.png".format(filename)
    plt.savefig(filepath, dpi = "figure", format = "png", bbox_inches = "tight", pad_inches = 0)
    # To avoid too much memory being used up, close the active pyplot window.
    plt.close()
    return None

Now, we shall create a function that exports `n_img` images from the Julia graph for $j_p = re^{ai}$, where $r$ is determined by the input of the function and $a$ is chosen at equally spaced intervals between $[0, 2\pi]$.

In [23]:
def julia_export(r, max_n, c_min, c_max, c_intervals, 
plt_width, plt_height, n_img, verbose = False):
    '''
    Exports n_img number of images for 
    a Julia set with parameter j_p = re^{ai} where a is chosen
    at n_img equally spaced intervals between [0, 2*pi]

    Inputs: 
    r - The r in re^{ai}
    max_n, c_min, c_max, c_intervals, plt_width, plt_height- as with julia_graphpic.
    n_img - number of different a considered in re^{ai}.
    '''
    n_intervals = np.pi*2 / (n_img - 1)
    for i in range(n_img):
        # Calculate a in between the intervals and calculate j_p.
        a = (i * n_intervals)
        j_p = r * np.exp(a * 1j)
        # Export an image of each Julia set.
        filename = "JuliaPic{}".format(i + 1)
        julia_graphpic(j_p, max_n, c_min, c_max, c_intervals, plt_width, plt_height, filename)
        # If verbose is True, output the current progress line as an addendum.
        if verbose:
            print("Please wait while the images are being exported. {}% of all the images has now been exported.".format(int((i + 1) / n_img * 100)), end = "\r")
    return None

Now, we shall create a `PyGame` animation for displaying the Julia graphs. Note that the additional argument `runfile` is added for easier testing purpose, if it was set to `False`, then it will display the animation using the preset `.jpg` files in the directory.

In [24]:
def julia_animation(folder_name, r = 0.7885, max_n = 100, c_min = -2, c_max = 2, 
c_intervals = 200, plt_width = 14, plt_height = 12, n_img = 200,
 screen_w = 800, screen_h = 600, verbose = True, runfile = True):
    '''
    Creates a PyGame animation with
    slides of images of the Julia graph with parameter j_p = re^{ai}, 
    where a is n_img - 1 equally spaced intervals between 0 and 2pi.
    The animation has a resolution width of screen_w pixels and height of
    screen_h pixels.
    '''
    if runfile:
        try:
            # If runfile is True, always export the figure to directory/JULIA_GRAPHS
            folder_name = 'JULIA_GRAPHS'
            julia_export(r, max_n, c_min, c_max, c_intervals, plt_width, plt_height, n_img, verbose)
            if verbose:
                print("\nAll the images has now been exported. Starting Program...")
        except:
            print("Error producing graphs!")
            return None
    screen_size = (screen_w, screen_h)
    white = (255, 255, 255)
    speed_factor = 1
    clock = pygame.time.Clock()
    # Initialise PyGame.
    pygame.init()
    if verbose:
        print("Program has now initialised!")
    # Set the screen size and the title of the program.
    screen = pygame.display.set_mode(screen_size)
    # Set the path of the Julia image and load the image to the correct size.
    currwd = os.getcwd()
    filename = str(currwd) + "//" + folder_name + "//JuliaPic1.png"
    julia = pygame.image.load(filename)
    julia = pygame.transform.scale(julia, screen_size)
    # Create a rectangle for the image to be loaded in.
    julia_rect = pygame.Rect(0, 0, screen_w, screen_h)
    # Fill the background white and initialise the the image into the program.
    screen.fill(white)
    screen.blit(julia, julia_rect)
    # Refresh the program display.
    pygame.display.flip()
    # Set initial values of variables for later.
    Running = True
    pause = True
    i = 0
    # Main Animation Loop
    while Running:
        # Events and User key press detection
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                Running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                pause = not pause
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                speed_factor = 0.75 * speed_factor
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                speed_factor = (4 / 3) * speed_factor
            title = f"Animation of Julia Sets of r = {r}"
            title += f" Current Speed: {int(speed_factor)}"
            title += "(Keystroke:  Space - Stop, Up/Down - Speed)"
            pygame.display.set_caption(title)
        if not pause:
            # Set the timimg between the transition of images.
            fps = 10 * speed_factor
            clock.tick(fps)
            i += 1
            if i == n_img:
                i = 0
            # Load the corresponding Julia Graphs
            currwd = os.getcwd()
            filename = str(currwd) + "//" + folder_name +"//JuliaPic{}.png".format(i + 1)
            julia = pygame.image.load(filename)
            # Resize the loaded Julia graph and update the display.
            julia = pygame.transform.scale(julia, screen_size)
            julia_rect = pygame.Rect(0, 0, screen_w, screen_h)
            screen.fill(white)
            screen.blit(julia, julia_rect)
            pygame.display.flip()
    pygame.quit()
    return None

We shall now use the default parameters of the `julia_animation` function, which will generate $200$ image files of the Julia sets with parameter $j_p = 0.7885e^{ai}$ where the numbers $a$ are chosen at equally spaced intervals in $[0, 2 \pi]$, and display these image files in a film like animation with the program resolution being $800$ pixels by $600$ pixels.

In [25]:
# Original export-then-run command, takes about 5 minutes
# julia_animation()
# Using exported graphs
julia_animation('JULIA_GRAPHS', r = 0.7885, n_img = 200, runfile= False)

Program has now initialised!


From experimenting with the parameter `r` in the `julia_animation` function, one can see that changing the value of $r$ in $j_p = re^{ai}$ changes the distance between multiple "clusters" in the image, and it is only when these "clusters" meet that a significant visual effect happens. Upon trial and error, another value of $r$ that creates striking effects is $0.66$, and from this, one can see that there are many values of $r$ that creates such visual effects.

In [26]:
# Original export-then-run command, takes about 30 minutes
# julia_animation('JULIA_GRAPHS', r = 0.66, n_img=400,c_intervals = 400)
# Using Exported graphs in JULIA_GRAPHS2
julia_animation('JULIA_GRAPHS2', r = 0.66, n_img = 400,runfile= False)

Program has now initialised!


The end result can be found at `Julia_Sample.mp4`.

<a id='S4.2'></a> 
#### Part 2.1.2 Animation of Mandlebrot Sets

Firstly, need to know as to what a Mandelbrot set is. The **Mandelbrot set** is defined as the values of $c$ for which the following recurrence relation is bounded for all $(z_n)_{n \geq 0}$:

$$
z_0 = 0  \quad \text{and} \quad z_n = z_{n-1}^2 + c, \quad \text{for } n = 1,2,3,\ldots
$$

We say that $(z_n)_{n \geq 0}$ is the Mandelbrot sequence induced by $c$. From this, we can see that $(z_n)_{n \geq 0}$ only relies on $c$.

In [27]:
def mandelbrot(c, max_n):
    '''
    The mandelbrot function calculates the largest term n_max
    for which each term z_n of the corresponding Mandelbrot 
    sequence induced by c is bounded by 2. 
    '''
    z =  0 + 0j
    n_max = 0
    # Iterate until term is not bounded by 2.
    while abs(z) <= 2:
        # Derive the next term of the sequence from the current term.
        z = (z ** 2) + c
        n_max += 1
        if n_max == max_n:
            break
    # Return the value of the largest number n_max.
    return n_max

We shall now create data for our Mandelbrot set. In order to do this, we need to create vectors `x_vector` and `y_vector` in order to create grid matrices `x_matrix` and `y_matrix` as input matricies. The `x_vector` must be a number `n_xpts` of equally spaced intervals between the values `x_min` and `x_max`, and the `y_vector` must be a number `n_ypts` of equally spaced intervals between the values `y_min` and `y_max`.

We shall create the following function that takes a number `n_xpts` of equally spaced intervals between the values `x_min` and `x_max`, and takes a number `n_ypts` of equally spaced intervals between the values `y_min` and `y_max` and calculates `x_matrix` and `y_matrix`, and inputs the matrices into the `mandelbrot` function to create an output matrix `z_matrix`. In order to generate data needed for the set, we need `x_matrix`, `y_matrix` and `z_matrix`.

In [28]:
def mandelbrot_data(max_n, x_min, x_max, n_xpts, y_min, y_max, n_ypts):
    '''
    The mandelbrot_data function calculates the matrices 
    required for graphing the mandelbrot function.
    '''
    # Calculate the vectors required for the two matrices required for input.
    x_vector = np.linspace(x_min, x_max, n_xpts)
    y_vector = np.linspace(y_min, y_max, n_ypts)
    x_matrix, y_matrix = np.meshgrid(x_vector, y_vector)
    # Create a zero output matrix with the same dimensions as the input matrices.
    x_matrix_rows, x_matrix_cols = np.shape(x_matrix)
    z_matrix = np.zeros((x_matrix_rows, x_matrix_cols), dtype = int)
    # Calculate every element of the output matrix for every 
    # element in the input matrix using the mandelbrot function.
    for i in range(x_matrix_rows):
        for j in range(x_matrix_cols):
            c = x_matrix[i, j] + y_matrix[i, j] * 1j
            z_matrix[i, j] = mandelbrot(c, max_n)
    return x_matrix, y_matrix, z_matrix

We shall create a folder, so that we can export the images into an appropriate directory entitled "MANDELBROT_GRAPHS".

In [29]:
# Obtain the current working directory.
currwd2 = str(os.getcwd())
# Create a subdirectory within the current working directory using the system terminal.
createdir2 = "mkdir \"{}//MANDELBROT_GRAPHS\"".format(currwd2)
os.system(createdir2)

1

We shall now create a function that generates and exports the Mandelbrot set.

In [30]:
def mandelbrot_graphpic(max_n, x_min, x_max, n_xpts, y_min, y_max, n_ypts, fig_w, fig_h, filename):
    '''
    The mandelbrot_graphpic function creates a Mandelbrot graph and exports
    the Mandelbrot graph as a picture with default resuloutions.
    
    '''
    # Calculate the two input matrices and one output matrix using the mandelbrot_data function.
    x_matrix, y_matrix, z_matrix = mandelbrot_data(max_n, x_min, x_max, n_xpts, y_min, y_max, n_ypts)
    # Produce a Mandelbrot graph from the given matrices without the axes.
    plt.figure(figsize = (fig_w, fig_h))
    plt.pcolor(x_matrix, y_matrix, z_matrix, cmap = 'plasma', vmin = 0, vmax = max_n, shading = "auto")
    plt.axis("off")
    # Create a filepath directory string and export  
    # the aforementioned Mandelbrot graph into the filepath directory.
    currwd = os.getcwd()
    filepath = str(currwd) + "//MANDELBROT_GRAPHS//{}.png".format(filename)
    plt.savefig(filepath, dpi = "figure", format = "png", bbox_inches = "tight", pad_inches = 0)
    plt.close()
    return None

Before we create a function that zooms in and out of a Mandelbrot set, we need to create a "recipe" for the function.

We shall now define as to what "clicking and dragging" the mouse is in terms of the actions.

> **Definition:** A **click and drag** of a mouse button consists of the following three actions in succession.
>
> 1. Pressing down on the left mouse button,
>
> 2. Moving the mouse, and
>
> 3. Releasing the left mouse button.
>
> All of these are stipulated by different events in PyGame.

Hence, for our program, the "recipe" for a "cropbox" is of the following, we initiate new boolean variable `drag` and `rext`:

> 1. If the user presses down the left mouse button, then set `drag` to be `True` and obtain position of the mouse (initial position).
>
> 2. If the user then moves the mouse around, and if `drag` is `True`, then set `rect` to be `True`.
> 
> 3. If `rect` is `True`, then obtain current poisition of the mouse whenever the mouse moves around and display an outline box based on the initial position of the mouse and the current position of the mouse.
>
> 4. If the user releases the mouse button, then set `rect` and `drag` to be `False`.

PyGame has no definitive function for a button, hence we shall need a "recipe" for the buttons in our program, and the "recipe" is of the following: Using a new boolean variable `button`,

> 1. Create a filled box with a shade of green, and position the box with appropriate co-ordinates.
>
> 2. If the user presses down the left mouse button within the co-ordinates of the button, set a variable `button` to be `True`, and update the colour of the button to be a lighter green.
>
> 3. Once the processes initiated by the user or by the button has been completed and the `button` variable is set to `False`, the colour of the button will go back to the initial colour, that being green in this case.

Using the principles above, we shall now create a program that zooms in and out of the animation, and we shall call the associated function `mandelbrot_program`. Due to the textboxes from within the program, we must set the minimum compatibility resolution width as $680$ pixels.

In [31]:
def mandelbrot_program(max_n = 100, x_min = -2, x_max = 1, n_xpts = 400, 
y_min = -1.5, y_max = 1.5, n_ypts = 400, 
fig_w = 14, fig_h = 12, screen_w = 800, screen_h = 600):
    '''
    The mandelbrot_program function creates a PyGame/SDL program that
    displays a Mandelbrot graph, allowing the user to zoom in and out
    the graph. The animation has a resolution width of screen_w pixels
    and height of screen_h pixels.
    '''
    # Set the minimum width resolution for running the program.
    appcompatresx = 680
    if screen_w < appcompatresx:
        print("The width resolution you have inputted into the function is too small. You need a width resolution of at least {} pixels.".format(appcompatresx))
        return None
    # Try exporting the required images for the program.
    try:
        mandelbrot_graphpic(max_n, x_min, x_max, n_xpts, y_min, y_max, n_ypts, fig_w, fig_h, "MandelbrotPic0")
    except:
        print("Error producing graphs!")
        return None
    # Store the input screen size and the size of the rescaled image as tuples.
    screen_size = (screen_w, screen_h)
    mandelbrot_size = (screen_w, screen_h - 50)
    # Store colours as RGB tuples.
    white = (255, 255, 255)
    black = (0, 0, 0)
    grey = (119, 119, 119)
    green_notactive = (114, 179, 101)
    green_active = (161, 251, 142)
    red = (255, 0, 0)
    orange = (240, 134, 80)
    # Set graph axes limits as lists and add initial values.
    x_min_list = []
    x_max_list = []
    y_min_list = []
    y_max_list = []
    x_min_list.append(x_min)
    x_max_list.append(x_max)
    y_min_list.append(y_min)
    y_max_list.append(y_max)
    # Initialise PyGame and its fonts.
    pygame.init()
    pygame.font.init()
    # Set the screen size and the title of the program.
    screen = pygame.display.set_mode(screen_size)
    title = "Animation of Mandelbrot Sets"
    title += " "
    title += '(Keystroke:  \'Drag and release\' to zoom in if the functionality is enabled.)'
    pygame.display.set_caption(title)
    # Set the path of the Mandelbrot image and load the image to the correct size.
    currwd = os.getcwd()
    filename = str(currwd) + "//MANDELBROT_GRAPHS//MandelbrotPic0.png"
    mandelbrot = pygame.image.load(filename)
    mandelbrot = pygame.transform.scale(mandelbrot, mandelbrot_size)
    # Create and position initial rectangles for Mandelbrot images, drag boxes, and zoom buttons.
    mandelbrot_rect = pygame.Rect(0, 0, mandelbrot_size[0], mandelbrot_size[1])
    cropbox = pygame.Rect(0, 0, 0, 0)
    zoom_in_button_left = screen_size[0] / 2 - 115
    zoom_out_button_left = screen_size[0] - 115
    zoom_button_top = screen_size[1] - 45
    zoom_button_width = 80
    zoom_button_height = 40
    zoom_in_button = pygame.Rect(zoom_in_button_left, zoom_button_top, zoom_button_width, zoom_button_height)
    zoom_out_button = pygame.Rect(zoom_out_button_left, zoom_button_top, zoom_button_width, zoom_button_height)
    # Create Text object and other text messages
    text = pygame.font.SysFont("arial", 14)
    errortext = pygame.font.SysFont("arial", 18)
    text1 = text.render("Click the first button on the right", True, black)
    textbox1 = text1.get_rect()
    textbox_centre_x = zoom_in_button_left / 2
    textbox1_centre_y = screen_size[1] - 35
    textbox1.center = (textbox_centre_x, textbox1_centre_y)
    text2 = text.render("to enable the zoom in function.", True, black)
    textbox2 = text2.get_rect()
    textbox2_centre_y = screen_size[1] - 15
    textbox2.left = textbox1.left
    textbox2.centery = textbox2_centre_y
    text3 = text.render("to disable the zoom in function.", True, black)
    textbox3 = text3.get_rect()
    textbox3.left = textbox1.left
    textbox3.centery = textbox2_centre_y
    text4 = text.render("Click the button on the right to", True, black)
    textbox4 = text4.get_rect()
    textbox4.right = zoom_out_button_left - (zoom_in_button_left - textbox1.right)
    textbox4.centery = textbox1_centre_y
    text5 = text.render("zoom out to the previous image.", True, black)
    textbox5 = text5.get_rect()
    textbox5.left = textbox4.left
    textbox5.centery = textbox2_centre_y
    text6 = text.render("Please wait while", True, black)
    textbox6 = text6.get_rect()
    textbox6.center = textbox4.center
    text7 = text.render("it is zooming out.", True, black)
    textbox7 = text7.get_rect()
    textbox7.left = textbox6.left
    textbox7.centery = textbox5.centery
    # Set initial values of variables for later.    
    running = True
    drag = False
    rect = False
    zoomin_button = False
    zoomin = False
    zoomout_button = False
    zoomout = False
    i = 0
    # If running is set to True, continue the program.
    while running:
        # Obtain the events from system logs.
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            # If user down clicks the left mouse button, then look for position and check status of buttons.
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if event.button == 1:
                    if event.pos[1] <= mandelbrot_size[1] and zoomin_button:
                        drag = True
                        mouse_x1_pos = event.pos[0]
                        mouse_y1_pos = event.pos[1]
                    # If the position of the down click is within region of the
                    # buttons, then set or change the activity statuses of the buttons.
                    elif event.pos[0] >= zoom_in_button_left and event.pos[0] <= (zoom_in_button_left + zoom_button_width) and event.pos[1] >= zoom_button_top and event.pos[1] <= (zoom_button_top + zoom_button_height):
                        zoomin_button = not zoomin_button
                    elif event.pos[0] >= zoom_out_button_left and event.pos[0] <= (zoom_out_button_left + zoom_button_width) and event.pos[1] >= zoom_button_top and event.pos[1] <= (zoom_button_top + zoom_button_height):
                        zoomout_button = True
            # If user releases the left click button, then set variables where necessary.      
            elif event.type == pygame.MOUSEBUTTONUP:
                if event.button == 1:
                    drag = False
                    if zoomin_button and rect:
                        zoomin = True
                    elif zoomout_button:
                        zoomout = True
                    rect = False
            # If the zoom in funtionality is enabled and dragging is enabled,
            # and the mouse is moving, then obtain the position of the mouse 
            # and set Rect as True.
            elif event.type == pygame.MOUSEMOTION and drag and zoomin_button:
                (mouse_x2_pos, mouse_y2_pos) = pygame.mouse.get_pos()
                rect = True
        # Fill the background with white.
        screen.fill(white)
        # Depending on the activity status of the buttons, draw
        # the appropriate textboxes and buttons onto the screen.
        screen.blit(text1, textbox1)
        if zoomin_button:
            screen.blit(text3, textbox3)
            pygame.draw.rect(screen, green_active, zoom_in_button)
        else:
            screen.blit(text2, textbox2)
            pygame.draw.rect(screen, green_notactive, zoom_in_button)
        if zoomout_button:
            pygame.draw.rect(screen, green_active, zoom_out_button)
        else:
            screen.blit(text4, textbox4)
            screen.blit(text5, textbox5)
            pygame.draw.rect(screen, green_notactive, zoom_out_button)
        # If rect is True, then calculate the appropriate position and 
        # size of the rectangle and draw the rectangle onto the screen.
        if rect:
            rect_width = abs(mouse_x2_pos - mouse_x1_pos)
            if mouse_y2_pos <= mandelbrot_size[1]:
                rect_height = abs(mouse_y2_pos - mouse_y1_pos)
            rect_left = min(mouse_x1_pos, mouse_x2_pos)
            rect_top = min(mouse_y1_pos, mouse_y2_pos)
            cropbox = pygame.Rect(rect_left, rect_top, rect_width, rect_height)
            screen.blit(mandelbrot, mandelbrot_rect)
            pygame.draw.rect(screen, grey, cropbox, 2)
            pygame.display.flip()
        else:
            # If zoomin is True, then start zooming in.
            if zoomin:
                # Display the appropriate message onto the screen.
                textwait = errortext.render("Please wait while the image is zooming in...", True, white, orange)
                textboxwait = textwait.get_rect()
                textboxwait_centre_x = mandelbrot_size[0] / 2
                textboxwait_centre_y = mandelbrot_size[1] / 2
                textboxwait.center = (textboxwait_centre_x, textboxwait_centre_y)
                screen.blit(mandelbrot, mandelbrot_rect)
                # Display the current rectangle onto the screen.
                pygame.draw.rect(screen, grey, cropbox, 2)
                screen.blit(textwait, textboxwait)
                pygame.display.flip()
                # Linearly interpolate the screen resolution of the rectangle against the axes of
                # the corresponding Mandelbrot graph, and add the new axes limits to the list.
                y_diff = y_max - y_min
                x_diff = x_max - x_min
                y_max_old = y_max
                x_min_old = x_min
                y_max = y_max_old - ((rect_top / mandelbrot_size[1]) * y_diff)
                rect_bottom = rect_top + rect_height
                y_min = y_max_old - ((rect_bottom / mandelbrot_size[1]) * y_diff)
                x_min = x_min_old + ((rect_left / mandelbrot_size[0]) * x_diff)
                rect_right = rect_left + rect_width
                x_max = x_min_old + ((rect_right / mandelbrot_size[0]) * x_diff)
                x_min_list.append(x_min)
                x_max_list.append(x_max)
                y_min_list.append(y_min)
                y_max_list.append(y_max)
                # Store the Mandelbrot graph with new axes limits
                # into a new image file, and display the new image.
                i += 1
                mandelbrot_graphpic(max_n, x_min, x_max, n_xpts, y_min, y_max, n_ypts, fig_w, fig_h, "MandelbrotPic{}".format(i))
                filename = str(currwd) + "//MANDELBROT_GRAPHS//MandelbrotPic{}.png".format(i)
                mandelbrot = pygame.image.load(filename)
                mandelbrot = pygame.transform.scale(mandelbrot, mandelbrot_size)
                screen.blit(mandelbrot, mandelbrot_rect)
                pygame.display.flip()
                # Clear the system events log to avoid bugs.
                pygame.event.clear()                
                # Set zoomin to False.
                zoomin = False
            elif not zoomout:
                # If zoomin and zoomout is False, then draw 
                # the current Mandlebrot graph onto the screen.
                screen.blit(mandelbrot, mandelbrot_rect)
                pygame.display.flip()
            # If zoomout is True, then start zooming out where necessary.
            if zoomout:
                # Check image index.
                if i <= 0:
                    # Display an error message for one second when user has zoomed out to the maximum.
                    texterror1 = errortext.render("ERROR: You have zoomed out to the original image!", True, white, red)
                    textboxerror1 = texterror1.get_rect()
                    textboxerror1_centre_x = mandelbrot_size[0] / 2
                    textboxerror1_centre_y = mandelbrot_size[1] / 2
                    textboxerror1.center = (textboxerror1_centre_x, textboxerror1_centre_y)
                    screen.blit(mandelbrot, mandelbrot_rect)
                    screen.blit(texterror1, textboxerror1)
                    pygame.display.flip()
                    zoomout = False
                    zoomout_button = False
                    pygame.time.delay(1000)
                    pygame.event.clear()
                else:
                    # Display the appropriate message onto the screen.
                    i -= 1
                    screen.blit(text6, textbox6)
                    screen.blit(text7, textbox7)
                    screen.blit(mandelbrot, mandelbrot_rect)
                    pygame.display.flip()
                    # Display the previous zoomed out image, and go back to the previous axes limits.
                    filename = str(currwd) + "//MANDELBROT_GRAPHS//MandelbrotPic{}.png".format(i)
                    mandelbrot = pygame.image.load(filename)
                    mandelbrot = pygame.transform.scale(mandelbrot, mandelbrot_size)
                    x_min = x_min_list[i]
                    x_max = x_max_list[i]
                    y_min = y_min_list[i]
                    y_max = y_max_list[i]
                    x_min_list.pop()
                    x_max_list.pop()
                    y_min_list.pop()
                    y_max_list.pop()
                    # Clear the system events log to avoid bugs.
                    pygame.event.clear()
                    # Set zoomout to False, and "unclick" the zoom out button.
                    zoomout = False
                    zoomout_button = False
    # Close the PyGame window.
    pygame.quit()
    return None

We shall now call the function with the default variables. This in turn produces a program with resolution of $800$ pixels by $600$ pixels, with the initial Mandelbrot set within the region $[-2, 1] \times [-1.5, 1.5]$. 

In [32]:
mandelbrot_program()

<a id='S5'></a>
### Section 2.2: Tower of Hanoi 

The Tower of Hanoi is a famous mathematical puzzle dating back to the $19^{th}$ century (see the introductory chapter of [(Hinz, Klavzar and Petr, 2018)](#HKP2018)). We have three poles/pegs and $n$ discs/disks that fit onto the poles. The discs differ in size and are initially arranged in a stack on one of the poles, in order from the largest disc (disc $n$) at the bottom to the smallest disc (disc $1$) at the top.

The problem is to move the stack of discs from one pole to another pole while obeying the following rules: 
- Move only one disc at a time.
- Never place a disc on top of one smaller than it.

In this section, we will look at a way to solve and animate the puzzles, as well as more restrictions, variants, and limitations of our code.

<a id='S5.1'></a> 
#### Part 2.2.1 Definitions and Objectives


In this part, we shall look at the necessary definitions and code to describe our objectives accurately in terms of the output of the code.

Firstly, we need to make some key definitions. Note that we shall call the pegs from left to right pegs $A$, $B$ and $C$ respectively.


> **Definition**: In the puzzle, A **move** consists of the following two pieces of information:
>
> •	A positive integer $d$ as the disc (of size $d$) being moved, and
>
> •	An indicator of the direction of the move, for which the values can only be Left or Right.

> **Definition**: A **peg** is a list of discs. A **configuration** (of the pegs) is the ordered triple $T =  (A, B, C)$ of the pegs mentioned thereof.

Remark: We shall explain further what an "indicator" means in this context when we implement the class `Move` later.


Next, we need to know mathematically what a puzzle is.

> **Definition**: A **puzzle** $P$ consists of the following three pieces of information:
>
> •	A configuration called an **initial configuration** $T_0$,
>
> •	A configuration called an **objective configuration** $T_f$, and
>
> •	A set of Rules $R$.

Different variants of the Towers of Hanoi has different initial configurations, objective configurations, and rules.


We shall now introduce a rigorous definition of solving a puzzle.

> **Definition**: Given a puzzle, a move $M$ and a configuration $T$, $M$ is **a valid move of $T$ under $P$** if the rules $R$ in $P$ are followed.
>
> For such a move, the configuration $T$ shall be changed accordingly to T′, which we shall call $T’$ as **the configuration attained by $M$ on $T$** (under $P$).

> **Definition**: For a positive integer n, a **solution for the puzzle $P$ with $n$ disks** is a sequence of moves $<M_1, M_2, …, M_s>$ such that the following holds under $P$:
>
> 1. $T_i$ is the configuration attained by $M_i$ on $T_{i-1}$ ,
> 2. $M_i$ is a valid move of $T_{i - 1}$,
> 3. $T_s$ is the objective configuration of the puzzle $T_f$,
>
> for all $1 ≤ i ≤ s$. We shall call $s$ the **length** of the solution. This gives rise to a configuration list under a solution for the puzzle $<T_1, T_2, …, T_s>$.


Using the terminology, we can state our objectives rigorously.

> Objective: Given the number of discs $n$, we should find a solution and a configuration list of the following puzzles:
>
> •	$P$, the Classical Tower of Hanoi Problem,
>
> •	$P$, the No Wrap Variant Problem, where wrap-around is prohibited, and
>
> •	$P$, the Bicolor Variant Problem, where the task is to separate 2 colours of discs.

Note that the output, namely the solution and the configuration list, shall be useful when we animate the problem in [Part 2.2.4](#S5.4).

It would be convenient for us to define $M$ and $T$ as a type of their own, as it would be easier to produce solutions and configuration lists in Python in that we can realize them as a `list` of a particular type.


We shall initialise $T$ as a list of lists. The structure of the tower is also noticeably clear from this. Lists are also mutable; hence it shall be easier for us to change the content later.

In [33]:
A = list(range(1, 7))
B = []
C= []
Tow = [A, B, C]
print(Tow)

[[1, 2, 3, 4, 5, 6], [], []]


In contrast, a move does not have to be mutable. Thus, it would be beneficial to exploit the object-oriented programming aspect of Python, which is well-explained in their documentation [(Python Software Foundation, 2022)](#PSF2022). Abiding by the definition, we shall define a new class `Move` that contains the two aspects: `disk` and `direction`.

In [34]:
class Move(object):
    """
    The class Move as introduced in the document. 
    We interpret this as a move in the towers of Hanoi.
    The key attribute of this class is Move.disk and Move.Direction, as introduced.
    """
    def __init__(self, dn=0, dir_n='Left'):
        self.disk = dn
        self.direction = dir_n
    # The Disk Number - Validation 
    def get_disk(self):
        return self._disk
    def set_disk(self, dn):
        error_message_disk = "The Disk is a Positive Integer"
        if not isinstance(dn, int):
            raise TypeError(error_message_disk)
        elif dn < 0:
            raise TypeError(error_message_disk)
        else:
            self._disk= dn
    disk = property(get_disk, set_disk)
    # The Move Direction -  Validation 
    def get_direction(self):
        return self._direction
    def set_direction(self, dir_n):
        error_messagge_direction = "The Direction is either the string 'Left' or 'Right'"
        if not isinstance(dir_n, str):
            raise TypeError(error_messagge_direction)
        elif dir_n != 'Left' and dir_n != 'Right':
            raise TypeError(error_messagge_direction)
        else:
            self._direction = dir_n
    direction = property(get_direction, set_direction)
    # Print out the move using __str__ method
    def __str__ (self):
        return f"Move disk {self.disk} to the {self.direction}"
    # Define the Equality of a Move
    def __eq__(self, other):
        if not isinstance(other, Move):
            return NotImplemented
        return self.disk == other.disk and self.direction == other.direction

In [35]:
# Example: Explore the Use of the class Move
# Declare a Move
M1 = Move()
M1.direction = "Right"
M1.disk = 5
# Print out a Move
print(M1)
# Another way to Declare a Move
M2 = Move(3, "Left")
# Accessing the Variable 
print(M2.disk)
print(M2.direction)
# Note: We can integrate f-strings in classes as follows:
print(f"{M1} is a good move!")
print(f"Do we want to {M2}?")
# We can test whether two moves are equal,
# obviously, they are equal iff they contain the same disk and direction
print(M1 == M2)

Move disk 5 to the Right
3
Left
Move disk 5 to the Right is a good move!
Do we want to Move disk 3 to the Left?
False


It is now easier to determine the following types of solutions that we are looking for:

-	A Solution: A `list` of Type `Move`, and

-	A configuration list under a solution: a `list` of type `int list X int list X int list`, where `X` denotes the Cartesian product.

From now on, we shall assume that everything shall be of the right type. That is, $n$ shall automatically denote the number of discs by default, and a solution and configuration shall always have the aforementioned type. This is a general programming concept called *duck typing*, and it stems from the following claim:

> If something walks and quacks like a duck, then it must be a duck. 
>
> [(“Understanding Interfaces in Go – Duncan Leung”)](#Leung2021)

This is a *pythonic* design, for which the concept is explained in their documentation in (Python Software Foundation, 2001).

<a id='S5.2'></a> 
#### Part 2.2.2 Producing a Solution

Using the above, we shall create a function that solves the puzzles using a powerful technique called **mutual recursion**. We have seen a lot of examples of simple recursion in lectures, but to produce the required solution for this case, we need to harness the power of mutual recursion. That is, we use multiple functions that recursively call each other.

Mutual recursion is a powerful idea, and there are many practical applications. One can see [(Rubio-Sánchez, Urquiza-Fuentes and Pareja-Flores, 2008)](#RUP2008) for more examples.

In particular, the following pseudocode illustrates the idea. Note that `MoveLeft(n)` produces the solution we need since the objective of the puzzle is to move the discs from peg A to C.

```
DEFINE MoveLeft(n): \\ Solution for the classical Puzzle - Move n discs from peg A to C
    if n = 1 \\ Base Case
        return [Move (1, 'Left')]
    else:
        \\ Perform the Following 3 Steps in order:
        1. MoveRight(n-1) \\ Move the first n-1 discs from peg A to B
        2. Move(n, 'Left') \\ Move disc n from peg A to C
        3. MoveRight(n-1) \\ Move the first n-1 discs from peg B to Peg C
END

DEFINE MoveRight(n): \\ Solution for moving n discs from peg C to A
    if n = 1 \\ Base Case
        return [Move(1, 'Right')]
    else:
        \\ Perform the Following 3 Steps in order:
        1. MoveLeft(n-1) \\ Move the first n-1 discs from peg A to C
        2. Move(n, 'Right') \\ Move disc n from peg A to B
        3. MoveLeft(n-1) \\ Move the first n-1 discs from peg C to Peg B
END
```

As seen, the function `MoveLeft` and `MoveRight` recursively call each other, this is an idea we shall keep on using. Now we shall see how it is implemented in Python.

In [36]:
ToH_Basic_Dict = {(1, 'Left'): [Move(1, 'Left')], (1, 'Right'): [Move(1, 'Right')]}
def ToH_Move_Left(n):
    """
    Given input n, produce the solution for the Classical Tower of Hanoi Problem
    """
    global ToH_Basic_Dict
    if (n, 'Left') not in ToH_Basic_Dict:
        MiddleMove = Move(n, 'Left')
        ToH_Basic_Dict[(n, 'Left')] = ToH_Move_Right(n-1) + [MiddleMove] + ToH_Move_Right(n-1)
    return ToH_Basic_Dict[(n, 'Left')]
def ToH_Move_Right(n):
    """
    A Supplmentary function to aid recursive call in ToH_Move_Left(n).
    """
    global ToH_Basic_Dict
    if (n, 'Right') not in ToH_Basic_Dict:
        MiddleMove = Move(n, 'Right')
        ToH_Basic_Dict[(n, 'Right')] = ToH_Move_Left(n-1) + [MiddleMove] + ToH_Move_Left(n-1)
    return ToH_Basic_Dict[(n, 'Right')]

Note that, other than the recursive call, memoization is implemented. This drastically improves the running time of the command, especially on large $n$. We will look at the space/time complexity issue in the [last part](#S5.5).

Now let us see the solution to the puzzle with $n = 4$:


In [37]:
print(ToH_Move_Left(4))

[<__main__.Move object at 0x00000167CE9A2970>, <__main__.Move object at 0x00000167CE9D8550>, <__main__.Move object at 0x00000167CE9A2970>, <__main__.Move object at 0x00000167CE9D86D0>, <__main__.Move object at 0x00000167CE9A2970>, <__main__.Move object at 0x00000167CE9D8550>, <__main__.Move object at 0x00000167CE9A2970>, <__main__.Move object at 0x00000167CE9D86A0>, <__main__.Move object at 0x00000167CE9A2970>, <__main__.Move object at 0x00000167CE9D8550>, <__main__.Move object at 0x00000167CE9A2970>, <__main__.Move object at 0x00000167CE9D86D0>, <__main__.Move object at 0x00000167CE9A2970>, <__main__.Move object at 0x00000167CE9D8550>, <__main__.Move object at 0x00000167CE9A2970>]


This is not particularly useful, because every element had been masked as an object. In order to see the moves, we can use a for loop:

In [38]:
for move in ToH_Move_Left(4):
    print(move)

Move disk 1 to the Right
Move disk 2 to the Left
Move disk 1 to the Right
Move disk 3 to the Right
Move disk 1 to the Right
Move disk 2 to the Left
Move disk 1 to the Right
Move disk 4 to the Left
Move disk 1 to the Right
Move disk 2 to the Left
Move disk 1 to the Right
Move disk 3 to the Right
Move disk 1 to the Right
Move disk 2 to the Left
Move disk 1 to the Right


So, we have produced our solution for the classical puzzle as required. We can also investigate the length of the solution:

In [39]:
for i in range(3, 7):
    print(f'for {i} disks, length of solution is {len(ToH_Move_Left(i))}')


for 3 disks, length of solution is 7
for 4 disks, length of solution is 15
for 5 disks, length of solution is 31
for 6 disks, length of solution is 63


It is not hard to show the (optimal) length for this classical version of the puzzle is always $2^n - 1$. This is because the length $T(n)$ always follow the recursive relation:
$$\begin{cases}T(1) = 1 \\ T(n) = T(n-1) + 1 + T(n-1) = 2T(n-1)+1\end{cases}$$

Note that how the recursive relation above closely resembles the implementation above. Solving it using any techniques we can get $T(n) = 2^n - 1$ (it can also be readily proven from mathematical induction).

This actually means as $n$ grows, the length of the list called by `ToH_Move_Left(n)` increases exponentially. This will quickly causes space issue when $n > 16$ . But as we can see in the [later sections](#S5.5), even with an exponentially long list, the running time can stay rather constant with memoization.


We shall now look at different variants of the puzzle. Firstly, we shall deal with the variant where no wrapping is allowed. Another equivalent way of forming this puzzle would be considering the puzzle as the classical variation of the Tower of Hanoi puzzle, with an additional rule that *every move must involve peg $B$*.

We shall also handle the problem with mutual recursion with a pseudocode algorithm; this one is particularly inspired by the discussion at [(Chen, 2015)](#Chen2015).

```
DEFINE NoWrap_Left(n): \\ Solution for the puzzle - Move n discs from peg A to C
    if n = 1: 
        \\ Perform the following 2 steps in order:
        1. Move(1, 'Right') \\ move disc from A to B
        2. Move(1, 'Right') \\ move disc from B to C
    else:
        \\ Perform the 5 Steps in Order:
        1. NoWrap_Left(n) \\ move n-1 from A to C 
        2. Move(n, 'Right') \\ move disc n from A to B
        3. NoWrap_Right(n) \\ move n-1 from C to A 
        4. Move(n, 'Right') \\ move disc n from B to C
        5. NoWrap_Left(n) \\move n-1 from A to C 
END    

DEFINE NoWrap_Right(n): \\ Solution for Moving n discs from peg C to A
    if n = 1:
        \\ Perform the following 2 steps in order:
        1. Move(1, 'Left') \\ move disc from C to B
        2. Move(1, 'Left') \\ move disc from B to A
    else:
        \\ Perform the 5 Steps in Order:
        1. NoWrap_Right(n) \\ move n-1 from C to A 
        2. Move(n, 'Left') \\ move disc n from C to B
        3. NoWrap_Left(n) \\ move n-1 from A to C 
        4. Move(n, 'Left') \\ move disc n from B to A
        5. NoWrap_Right(n) \\move n-1 from C to A 
END
```
The implementation is analogous to the classical variant of the puzzle. We shall also have a look at the solution and its length.


In [40]:
ToH_NoWrap_Dict = {
    (1, 'Left'): [Move(1, 'Right'), Move(1, 'Right')],
    (1, 'Right'): [Move(1, 'Left'), Move(1, 'Left')]
    }
# Left: A -> C, Right C -> A
def ToH_NoWrap_Move_Left(n):
    """
    Given input n, produce the solution for the No Wrap Variant.
    """
    global ToH_NoWrap_Dict
    if (n, 'Left') not in ToH_NoWrap_Dict:
        MiddleMove = Move(n, 'Right')
        ToH_NoWrap_Dict[(n, 'Left')] = ToH_NoWrap_Move_Left(n-1) + [MiddleMove] +\
             ToH_NoWrap_Move_Right(n-1) + [MiddleMove] +\
                  ToH_NoWrap_Move_Left(n-1)
    return ToH_NoWrap_Dict [(n, 'Left')]
        
def ToH_NoWrap_Move_Right(n):
    """
    A Supplmentary function to aid recursive call in ToH_NoWrap_Move_Left(n).
    """
    global ToH_NoWrap_Dict
    if (n, 'Right') not in ToH_NoWrap_Dict:
        MiddleMove = Move(n, 'Left')
        ToH_NoWrap_Dict[(n, 'Right')] = ToH_NoWrap_Move_Right(n-1) + [MiddleMove]  +\
    ToH_NoWrap_Move_Left(n-1) + [MiddleMove] +  ToH_NoWrap_Move_Right(n-1)
    return ToH_NoWrap_Dict [(n, 'Right')]

In [41]:
for i in range(3, 7):
    print(f'for {i} disks, length of solution is {len(ToH_NoWrap_Move_Left(i))}')

for 3 disks, length of solution is 26
for 4 disks, length of solution is 80
for 5 disks, length of solution is 242
for 6 disks, length of solution is 728


And, in fact, we can show that the (optimal) length of this solution is $3^n -1$. The intuition would be that we use one extra move in every iteration to accommodate the fact that it must pass through peg B.





Finally, we shall deal with the bicolour variant. Note that this implementation is not optimal, as we would see in the [next part](#S5.3).

The strategy here is to first move everything to peg $B$ and then redistribute it after changing the bottom two discs. Again, we shall start by introducing pseudocode:

```
DEFINE ToCenter(n): \\ Solution for Moving n discs to peg B
    if n = 1:  Move(1, 'Right') 
    Elif n = 2: [Move(2, 'Left'), Move(1, 'Right')]
    else:
        if n is even:
            \\ Perform the 4 Steps in Order:
            1. ToCenter(n-2) \\ move n-2 discs to peg B
            2. Left(n-2) \\ move n-2 discs from B to A
            3. Move(n, 'Left') \\ move n from C to B 
            4. Right(n-1) \\move n-1 discs from A to B 
        else: \\ if n is odd
            \\ Perform the 4 Steps in Order:
            1. ToCenter(n-1) \\ move n-1 discs to peg B
            2. Right(n-1) \\ move n-1 discs from B to C
            3. Move(n, 'Right') \\ move n from A to B 
            4. Left(n-1) \\move n-1 discs from C to B 
END    

DEFINE Solve_Bicolor(n): \\ Solution for the Bicolor Variant
    if n is not even, RAISE error
    if n = 2: [Move(2, 'Left'), Move(1, 'Left'), Move(2, 'Left')]
    else:
        \\ Perform the 3 Steps in Order:
        1. ToCenter(n-1) \\ move n-1 discs to peg B
        2. Move(n, 'Left') \\ move disc n from C to A
        3. REVERSE ToCenter(n) \\ redistribute the n-1 discs to their opposite pegs
END
```

Note that, for the last step, we shall need to reverse the whole list. In this case, we shall need to create a new function `rev_list` to reverse a list.

In [42]:
def rev_list(l):
    """
    Given a List l, return the reverse list of l.
    """
    ans = []
    for ele in reversed(l):
        ans += [ele]
    return ans

ToH_Center_Dict = {
    1: [Move(1, 'Right')],
    2: [Move(2, 'Left'), Move(1, 'Right')]
}
def ToH_To_Center(n):
    """
    A Supplmentary function to aid recursive call in ToH_BiColor_Solve(n).
    """
    global ToH_Center_Dict
    # For inducitve case, we know about To Center n-2 and n-1
    if n not in ToH_Center_Dict:
        if n % 2 == 0: # Even number, deal with the last two:
            MiddleMove = Move(n, 'Left')
            ToH_Center_Dict[n]  = ToH_To_Center(n-2) + ToH_Move_Left(n-2) + [MiddleMove] +ToH_Move_Right(n-1)
        else: #Odd number 
            MiddleMove = Move(n, 'Right')
            ToH_Center_Dict[n] = ToH_To_Center(n-1) + ToH_Move_Right(n-1) + [MiddleMove] +ToH_Move_Left(n-1)
    return ToH_Center_Dict[n]
ToH_BiColor_Dict = {
    2: [Move(2, 'Left'), Move(1, 'Left'), Move(2, 'Left')]
}
def ToH_Bicolor_Solve(n):
    """
    Given input n, produce the solution for the Bicolor Variant.
    """
    global ToH_BiColor_Dict
    even_error = 'n must be an even number!'
    if n % 2 != 0:
        print(even_error)
    elif n not in ToH_BiColor_Dict:
        MiddleMove = Move(n, 'Right')
        FirstMove = ToH_To_Center(n-1)
        ToH_BiColor_Dict[n] = FirstMove + [MiddleMove] + rev_list(FirstMove)
    return ToH_BiColor_Dict[n]

In [43]:
for i in range(2, 12, 2):
    print(f'for {i} disks, length of solution is {len(ToH_Bicolor_Solve(i))}')

for 2 disks, length of solution is 3
for 4 disks, length of solution is 19
for 6 disks, length of solution is 89
for 8 disks, length of solution is 375
for 10 disks, length of solution is 1525


<a id='S5.3'></a> 
#### Part 2.2.3 Identifying Valid Moves

In this part, we shall introduce the function `ToH_Valid_Move` in order to detect whether a move $M$ is a valid move of $T$ under $P$. We shall then use it to return the configuration list under a solution. This will achieve the objective set in the first part of our discussion. From here, we shall also present an interactive version of the puzzle.


The key idea of the `ToH_Valid_Move` Command is as follows:

>1.	Given a move `m`, assign a direction of m (i.e., `m.direction`) as Left or Right with a direction number `dir_num`, with the value being `1` when the direction is Right, and `-1` otherwise.
> 2. Given the Move `m,` ensure that the disc m.disc is at the top of one of the pegs. Find the peg and call that peg `count`.
> 3. Calculate the value of `count + dir_num` and assign it to `tar`. This tells us where we can put the disc at the target configuration. If wrapping around is allowed, we can work modulo 3; otherwise `tar` can only be 0, 1 or 2.
>
> 4. Mutate the configuration such that it becomes the configuration attained by `m` on `config` if the boolean variable `Validmove` is true, otherwise, return the unchanged configuration.


In [44]:
def ToH_Valid_Move(m, config, WrapAround = True, Verbose = False):
    """
    Input: 
        - m (Move), 
        - config (A configuration)
    Optional Arguments: 
        - WrapAround (bool) - Whether Wrap Around is allowed
        - Verbose (bool) - Whether error messages are printed out
    Return:
        - If m is a valid move of config, return The configuration attained by m on config
        - Otherwise, return config.
    """
    # Initialize the variable count and tar
    count = 2
    tar = 2
    ValidMove = False
    # Extract information of the move
    dn = m.disk
    dir_n = m.direction
    # Assign direction number to the move
    if dir_n == 'Right':
        dir_num = 1
    else:
        dir_num = -1
    # Detect the target disk, and determine which peg to move from (count) and which peg to move to (tar)
    for count, peg in enumerate(config):
        if len(peg) != 0 and peg[0] == dn:
            if Verbose: print(f'Disk {dn} detected at peg {count +1}! Moving to the {dir_n}')
            trial = count + dir_num
            if WrapAround:
                ValidMove = True
                tar = trial % 3
                break
            elif trial in [0, 1, 2]:
                ValidMove = True
                tar = trial
                break
        else:
            if Verbose: print(f"Can't detect disk {dn} at the top!")
    # Try to move the disk 
    try:
        if ValidMove:
            if config[tar] and dn >= config[tar][0]:
                raise TypeError
            else:
                config[count].pop(0)
                config[tar].insert(0,dn)
        else:  
            print('Invalid move! Returning Original configuration.')
    except: 
        if Verbose: print('Invalid move! Returning Original configuration.')
    return [config[0], config[1], config[2]]

The key point here is point 4. Due to the process of mutating the original configuration `config`, it will change whenever we call the function, so we do not have to redefine it upon successive calls of the function. 

This is an immensely powerful design (in technical jargon, he variable `config` is *dynamic*) that will save us some lines when designing the interactive version of the puzzles. The following code block illustrates this idea.

In [45]:
# Test Cell: The ToH_Valid_Move Function is dynamic
A = [1, 2, 3]
B = []
C = []
Tow = [A, B, C]
M_1 = Move(1, 'Left')
M_2 = Move(2, 'Left')
print(ToH_Valid_Move(M_1, Tow, WrapAround = False, Verbose = True))
print(ToH_Valid_Move(M_1, Tow, WrapAround = True, Verbose = True))
# Note that how Tow changed to [[2, 3], [], [1]] instead of the original configuration
print(Tow)
print(ToH_Valid_Move(M_2, Tow, Verbose = True))
print(ToH_Valid_Move(M_1, Tow, WrapAround = False))

Disk 1 detected at peg 1! Moving to the Left
Can't detect disk 1 at the top!
Can't detect disk 1 at the top!
Invalid move! Returning Original configuration.
[[1, 2, 3], [], []]
Disk 1 detected at peg 1! Moving to the Left
[[2, 3], [], [1]]
[[2, 3], [], [1]]
Disk 2 detected at peg 1! Moving to the Left
Invalid move! Returning Original configuration.
[[2, 3], [], [1]]
[[2, 3], [1], []]


From here, we shall display the configuration list under a solution for the puzzle. We shall first start with the classical variant.

In [46]:
def ToH_ConList_Display(n):
    """
    Given number of disks n,
    Print out the configuration list under a solution for the classical variant And return None
    """
    A = list(range(1, n+1))
    B = []
    C = []   
    TOW = [A, B, C]
    MoveList = ToH_Move_Left(n)
    print("Let's Start! Our Tower looks like:")
    print(str(TOW))
    for step_num, move in enumerate(MoveList):
        print(f'Step {step_num+1}: {move}')
        print(ToH_Valid_Move(move, TOW))
    return None
def ToH_NoWrap_ConList_Display(n):
    """
    Given number of disks n,
    Print out the configuration list under a solution for the No Wrap variant And return None
    """
    A = list(range(1, n+1))
    B = []
    C = []   
    TOW = [A, B, C]
    MoveList = ToH_NoWrap_Move_Left(n)
    print("Let's Start! Our Tower looks like:")
    print(str(TOW))
    for step_num, move in enumerate(MoveList):
        print(f'Step {step_num+1}: {move}')
        print(ToH_Valid_Move(move, TOW))
    return None

In [47]:
ToH_ConList_Display(4)

Let's Start! Our Tower looks like:
[[1, 2, 3, 4], [], []]
Step 1: Move disk 1 to the Right
[[2, 3, 4], [1], []]
Step 2: Move disk 2 to the Left
[[3, 4], [1], [2]]
Step 3: Move disk 1 to the Right
[[3, 4], [], [1, 2]]
Step 4: Move disk 3 to the Right
[[4], [3], [1, 2]]
Step 5: Move disk 1 to the Right
[[1, 4], [3], [2]]
Step 6: Move disk 2 to the Left
[[1, 4], [2, 3], []]
Step 7: Move disk 1 to the Right
[[4], [1, 2, 3], []]
Step 8: Move disk 4 to the Left
[[], [1, 2, 3], [4]]
Step 9: Move disk 1 to the Right
[[], [2, 3], [1, 4]]
Step 10: Move disk 2 to the Left
[[2], [3], [1, 4]]
Step 11: Move disk 1 to the Right
[[1, 2], [3], [4]]
Step 12: Move disk 3 to the Right
[[1, 2], [], [3, 4]]
Step 13: Move disk 1 to the Right
[[2], [1], [3, 4]]
Step 14: Move disk 2 to the Left
[[], [1], [2, 3, 4]]
Step 15: Move disk 1 to the Right
[[], [], [1, 2, 3, 4]]


We can deal with the no wrap variant and bicolor variant similarly.

In [48]:
def ToH_NoWrap_ConList_Display(n):
    """
    Given number of disks n,
    Print out the configuration list under a solution for the No Wrap variant.
    """
    A = list(range(1, n+1))
    B = []
    C = []   
    TOW = [A, B, C]
    MoveList = ToH_NoWrap_Move_Left(n)
    print("Let's Start! Our Tower looks like:")
    print(str(TOW))
    for step_num, move in enumerate(MoveList):
        print(f'Step {step_num+1}: {move}')
        print(ToH_Valid_Move(move, TOW))
    return None
def ToH_BiColor_ConList_Display(n):
    """
    Given number of disks n,
    Print out the configuration list under a solution for the bicolor variant.
    """
    A = list(range(1, n+1, 2))
    B = []
    C = list(range(2, n+1, 2))  
    TOW = [A, B, C]
    MoveList = ToH_Bicolor_Solve(n)
    print("Let's Start! Our Tower looks like:")
    print(str(TOW))
    for step_num, move in enumerate(MoveList):
        print(f'Step {step_num+1}: {move}')
        print(ToH_Valid_Move(move, TOW))
    return None

In [49]:
# No Wrap Variant
ToH_NoWrap_ConList_Display(3)

Let's Start! Our Tower looks like:
[[1, 2, 3], [], []]
Step 1: Move disk 1 to the Right
[[2, 3], [1], []]
Step 2: Move disk 1 to the Right
[[2, 3], [], [1]]
Step 3: Move disk 2 to the Right
[[3], [2], [1]]
Step 4: Move disk 1 to the Left
[[3], [1, 2], []]
Step 5: Move disk 1 to the Left
[[1, 3], [2], []]
Step 6: Move disk 2 to the Right
[[1, 3], [], [2]]
Step 7: Move disk 1 to the Right
[[3], [1], [2]]
Step 8: Move disk 1 to the Right
[[3], [], [1, 2]]
Step 9: Move disk 3 to the Right
[[], [3], [1, 2]]
Step 10: Move disk 1 to the Left
[[], [1, 3], [2]]
Step 11: Move disk 1 to the Left
[[1], [3], [2]]
Step 12: Move disk 2 to the Left
[[1], [2, 3], []]
Step 13: Move disk 1 to the Right
[[], [1, 2, 3], []]
Step 14: Move disk 1 to the Right
[[], [2, 3], [1]]
Step 15: Move disk 2 to the Left
[[2], [3], [1]]
Step 16: Move disk 1 to the Left
[[2], [1, 3], []]
Step 17: Move disk 1 to the Left
[[1, 2], [3], []]
Step 18: Move disk 3 to the Right
[[1, 2], [], [3]]
Step 19: Move disk 1 to the Righ

In [50]:
#Bicolor Variant
ToH_BiColor_ConList_Display(4)

Let's Start! Our Tower looks like:
[[1, 3], [], [2, 4]]
Step 1: Move disk 2 to the Left
[[1, 3], [2], [4]]
Step 2: Move disk 1 to the Right
[[3], [1, 2], [4]]
Step 3: Move disk 1 to the Left
[[1, 3], [2], [4]]
Step 4: Move disk 2 to the Right
[[1, 3], [], [2, 4]]
Step 5: Move disk 1 to the Left
[[3], [], [1, 2, 4]]
Step 6: Move disk 3 to the Right
[[], [3], [1, 2, 4]]
Step 7: Move disk 1 to the Right
[[1], [3], [2, 4]]
Step 8: Move disk 2 to the Left
[[1], [2, 3], [4]]
Step 9: Move disk 1 to the Right
[[], [1, 2, 3], [4]]
Step 10: Move disk 4 to the Right
[[4], [1, 2, 3], []]
Step 11: Move disk 1 to the Right
[[4], [2, 3], [1]]
Step 12: Move disk 2 to the Left
[[2, 4], [3], [1]]
Step 13: Move disk 1 to the Right
[[1, 2, 4], [3], []]
Step 14: Move disk 3 to the Right
[[1, 2, 4], [], [3]]
Step 15: Move disk 1 to the Left
[[2, 4], [], [1, 3]]
Step 16: Move disk 2 to the Right
[[4], [2], [1, 3]]
Step 17: Move disk 1 to the Left
[[4], [1, 2], [3]]
Step 18: Move disk 1 to the Right
[[4], [2]

As we can see, the answer of the configuration list indeed matches the definition and the objective configuration. Note that, in the `ToH_BiColor_Solve` function, the output is sub-optimal. For example, note that the output, shown below, after 4 steps,

```
[[1, 3], [], [2, 4]] <-- HERE
Step 1: move disc 2 to the Left
[[1, 3], [2], [4]]
Step 2: move disc 1 to the Right
[[3], [1, 2], [4]]
Step 3: move disc 1 to the Left
[[1, 3], [2], [4]]
Step 4: move disc 2 to the Right
[[1, 3], [], [2, 4]] <-- HERE
```

brings us back to the original configuration, so this does not need to be part of the solution.

From here, it is not hard for us to produce an interactive version of the three puzzles, where the user can feed in a move and play until the objective is reached.

In [51]:
def ToH_interactive():
    '''
    Allows an user to play the 3 different variants of the puzzle interactively.
    '''
    def ToH_Finish_Checker(Tow, Sol, Wrap = True):
        '''
        The main interactive element of the function.
        Inputs:
        Tow - Initial configuration 
        Sol - Objective Configuration
        Wrap - Whether wrap-around is allowed
        Exit the function call only if the user enters a move sequence that produces a solution.
        '''
        print('We start with the Configuration:')
        print(Tow)
        while Tow != Sol:
            dn = input('What will be the Disk you want to move? Please enter a positive integer: ')
            dir_n = input("What will be the direction? Please enter 'Left' or 'Right' without the quotes: ")
            trial_move = Move(int(dn), dir_n)
            ans = ToH_Valid_Move(trial_move, Tow, Wrap, Verbose = True)
            print(f'Now the Tower is: {ans}')
        print('You got to the solution! Well done.')
        return None
    GameEnd = False
    while not GameEnd:
        # Prompt User to choose a Mode 
        Mode = input('Which variant? c - classical, n - nowrap, b - bicolor, q- quit')
        try:
            # For each mode, initiate the correct initial and objective configuration, and run ToH_Finish_Checker.
            if Mode == 'c':
                n = input('start with how many disk? ')
                Init_config = [list(range(1, int(n)+1)), list(), list()]
                Obj_config = [list(), list(), list(range(1, int(n)+1))]
                ToH_Finish_Checker(Init_config, Obj_config)
                GameEnd = True
            elif Mode == 'n':
                n = input('start with how many disk? ')
                Init_config = [list(range(1, int(n)+1)), list(), list()]
                Obj_config = [list(), list(), list(range(1, int(n)+1))]
                ToH_Finish_Checker(Init_config, Obj_config, False)
                GameEnd = True
            elif Mode == 'b':
                n = input('start with how many disk? ')
                if int(n) % 2 != 0:
                    print('n must be an even number!')
                    GameEnd = True
                    break
                Init_config = [list(range(1, int(n)+1, 2)), list(), list(range(2, int(n)+1, 2))]
                Obj_config = [list(range(2, int(n)+1, 2)), list(), list(range(1, int(n)+1, 2))]
                ToH_Finish_Checker(Init_config, Obj_config)
                GameEnd = True
            elif Mode == 'q':
                print('Quitting! Bye.')
                GameEnd = True
            else:
                raise TypeError
        except:
            print('Invalid input!')

In [52]:
# Test: key press are c - 1 - 1 - Left
ToH_interactive()

Quitting! Bye.


<a id='S5.4'></a> 
#### Part 2.2.4 Animation of the Puzzles

In this part, we shall look as to how we can integrate the solution and configuration list such that the puzzles can be displayed as an animation.

First we shall create a background with a base and 3 pegs, just like the usual Tower of Hanoi setup.

In [53]:
def ToH_Draw_Background():
    """
    Draws the background of the Tower of Hanoi and keep it in the file ToH_BG.jpg.
    """
    screen_size = (1000, 600)
    black = (0, 0, 0)
    white = (255, 255, 255)
    grey = (100, 100, 100)
    pygame.init()
    screen = pygame.Surface(screen_size)
    screen.fill(white)
    base_rect = pygame.Rect(100,450,800,100)
    Peg1Start = (250, 150)
    Peg1End = (250, 450)
    Peg2Start = (500, 150)
    Peg2End = (500, 450)
    Peg3Start = (750, 150)
    Peg3End = (750, 450)
    pygame.draw.rect(screen, grey, base_rect)
    pygame.draw.line(screen, black, Peg1Start, Peg1End, width = 5)
    pygame.draw.line(screen, black, Peg2Start, Peg2End, width = 5)
    pygame.draw.line(screen, black, Peg3Start, Peg3End, width = 5) 
    pygame.image.save(screen, 'ToH_BG.jpg')
    return None

Please run this line of code to export `ToH_BG.jpg` in the directory.

In [54]:
ToH_Draw_Background()

Now, in order to access and update the background, we just need to use the `image`, `load` and `bilt` command in the `PyGame` module.

Note that we do not want to do the same and save discs as PNG files, and this is because they depend on the configuration, and they will be moved around in the animation.


Another hurdle to overcome is the fact that discs come in varied sizes. Usually, they are cylinders of the same height, but since we are looking at the puzzle from a side view, we shall treat the discs as rectangles with different widths but the same height.

In our implementation, the discs are labelled by the positive integers (which stemmed from the class `Move`), and we can set disc $1$ as $10$ pixels long, disc $2$ as $20$ pixels, ... et cetera.


If the disc size is fixed, where the discs are placed is uniquely determined by the configuration. We shall take advantage of this and write a function that, given the configuration, outputs where the discs should be drawn.



In [55]:
def ToH_Get_Coordinate(config):
    '''
    Given a configuration, return a list of coordinates where the disks are to be plotted.
    (TOP LEFT coordinate of a rectangle, each assume with width 15)
    '''
    ans = list()
    get_disk_length = lambda d_n: 10 * d_n
    for peg_num, peg in enumerate(config):
        base_x_coord = 250 * (peg_num+1)
        total_num_disk = len(peg)
        for count, disk_num in enumerate(peg):
            y_coord = 450 - (15*(total_num_disk - count))
            x_coord = base_x_coord-get_disk_length(disk_num)
            ans.append((x_coord, y_coord, disk_num))
    return ans

The `ToH_Get_Coordinate` acts as a helper to our animation functions, and we shall enumerate over the list to draw all the discs in a configuration.

In [56]:
def ToH_Basic(n, speed_factor):
    """
    Inputs: 
    - Number of Disks n 
    - The frame speed factor speed_factor
    Return: None
    Plays the animation of solving the Classic Variant of Tower of Hanoi.
    """
    # Basic initialization
    screen_size = (screen_width, screen_height)  = (1000, 600)
    white = (255,255,255)
    blue = (0, 0, 255)
    black = (0, 0, 0)
    frames_per_second = speed_factor
    clock = pygame.time.Clock()
    pygame.init()
    screen = pygame.display.set_mode(screen_size)
    # Draw the background
    bg = pygame.image.load("ToH_BG.jpg")
    bg_rect = pygame.Rect(0,0,screen_width,screen_height)
    # Draw the disks
    get_double_disk_length = lambda d_n: 20 * d_n
    Sol_Classic_Hanoi = ToH_Move_Left(n)
    len_move_list = len(Sol_Classic_Hanoi)
    Initial_Configuration = [list(range(1, n+1)), list(), list()]
    Current_Configuration = Initial_Configuration
    Start_Coord_List = ToH_Get_Coordinate(Initial_Configuration)
    for x_cor, y_cor, disk in Start_Coord_List:
        disk_rect = pygame.Rect(x_cor, y_cor, get_double_disk_length(disk), 15)
        pygame.draw.rect(bg, blue, disk_rect)
        pygame.draw.rect(bg, black, disk_rect, width = 3)
    # Initialization of animation
    # Index_move will track through the solution for the puzzle
    index_move = 0
    screen.fill(white)
    screen.blit(bg, bg_rect)
    pygame.display.flip()
    running = True
    run_anim = False
    # Main Loop
    while running:
        # Key Press Detecting
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                run_anim = not run_anim 
            # Speed change if ArrowUp/ArrowDown is pressed
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                frames_per_second = (4/3)* frames_per_second
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                frames_per_second = 0.75 * frames_per_second
            # Title bar
            caption = 'Tower of Hanoi - Classical Variant'
            caption += f' Speed = {int(frames_per_second)}             '
            caption += '(Keystroke:  \'Space\' to start or pause, Up/Down for speed adjusting))'
            pygame.display.set_caption(caption)
        if run_anim and index_move < len_move_list:
            screen.fill(white)
            # Background has changed, update it again
            bg = pygame.image.load("ToH_BG.jpg")
            # Extract Current Configuration
            current_move = Sol_Classic_Hanoi[index_move]
            Current_Configuration = ToH_Valid_Move(current_move,Current_Configuration)
            # Print Information
            my_font = pygame.font.SysFont('Comic Sans MS', 30)
            text_surface = my_font.render(f'Move {index_move+1}/{len_move_list}: {current_move}', False, (255, 0, 0))
            bg.blit(text_surface, (0, 0))
            # Update iterator
            index_move = (index_move + 1) 
            ans = ToH_Get_Coordinate(Current_Configuration)
            # Draw the new disks
            for x_cor, y_cor, disk in ans:
                disk_rect = pygame.Rect(x_cor, y_cor, get_double_disk_length(disk), 15)
                pygame.draw.rect(bg, blue, disk_rect)
                pygame.draw.rect(bg, black, disk_rect, width = 3)         
        # Re-initialise the display 
        screen.blit(bg, (15, 0))
        pygame.display.flip()
        screen.fill(white)
        clock.tick(frames_per_second) 
    pygame.quit()
    return None 

In [57]:
ToH_Basic(5,1)

(Note that the animation shows the configuration before applying the last move, thus it is easy to see that the animation produces the objective configuration.)

Now we shall discuss the key difference between implementing this and the other two variants of the puzzle.

1.	The No Wrap Variant is similar, except with WrapAround = False in the ToH_Valid_Move command.

2.	The Bicolor Variant: The initial configuration of the discs shall be changed accordingly. We shall also colour the discs red and green instead of blue for visual effects.


In [58]:
def ToH_NoWrap(n, speed_factor):
    """
    Plays the animation of solving the No Wrap Variant of Tower of Hanoi.
    Inputs: 
    - Number of Disks n 
    - The frame speed factor speed_factor
    """
    # Basic initialization
    screen_size = (screen_width, screen_height)  = (1000, 600)
    white = (255,255,255)
    blue = (0, 0, 255)
    black = (0, 0, 0)
    frames_per_second = speed_factor
    clock = pygame.time.Clock()
    pygame.init()
    screen = pygame.display.set_mode(screen_size)

    # Draw the background
    bg = pygame.image.load("ToH_BG.jpg")
    bg_rect = pygame.Rect(0,0,screen_width,screen_height)
    # Draw the disks
    get_double_disk_length = lambda d_n: 20 * d_n
    Sol_NoWrap_Hanoi = ToH_NoWrap_Move_Left(n)
    len_move_list = len(Sol_NoWrap_Hanoi)
    Initial_Configuration = [list(range(1, n+1)), list(), list()]
    Current_Configuration = Initial_Configuration
    Start_Coord_List = ToH_Get_Coordinate(Initial_Configuration)
    for x_cor, y_cor, disk in Start_Coord_List:
        disk_rect = pygame.Rect(x_cor, y_cor, get_double_disk_length(disk), 15)
        pygame.draw.rect(bg, blue, disk_rect)
        pygame.draw.rect(bg, black, disk_rect, width = 3)
    # Initialization of animation
    # Index_move will track through the solution for the puzzle
    index_move = 0
    screen.fill(white)
    screen.blit(bg, bg_rect)
    pygame.display.flip()
    running = True
    run_anim = False
    # Main Loop - Stop animating when the end of move list is reached
    while running:
        # Key Press Detecting
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                run_anim = not run_anim 
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                frames_per_second = (4/3)* frames_per_second
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                frames_per_second = 0.75 * frames_per_second
            # Title bar
            caption = 'Tower of Hanoi - No Wrapping Variant'
            caption += f' Speed = {int(frames_per_second)}             '
            caption += '(Keystroke:  \'Space\' to start or pause, Up/Down for speed adjusting))'
            pygame.display.set_caption(caption)
        if run_anim and index_move < len_move_list:
            screen.fill(white)
            # Background has changed, update it again
            bg = pygame.image.load("ToH_BG.jpg")
            # Extract Current Configuration
            current_move = Sol_NoWrap_Hanoi[index_move]
            Current_Configuration = ToH_Valid_Move(current_move,Current_Configuration, False)
            # Print Information
            my_font = pygame.font.SysFont('Comic Sans MS', 30)
            text_surface = my_font.render(f'Move {index_move+1}/{len_move_list}: {current_move}', False, (255, 0, 0))
            bg.blit(text_surface, (0, 0))
            # Update iterator
            index_move = index_move + 1
            ans = ToH_Get_Coordinate(Current_Configuration)
            # Draw the new disks
            for x_cor, y_cor, disk in ans:
                disk_rect = pygame.Rect(x_cor, y_cor, get_double_disk_length(disk), 15)
                pygame.draw.rect(bg, blue, disk_rect)
                pygame.draw.rect(bg, black, disk_rect, width = 3)
        # Re-initialise the display 
        screen.blit(bg, (15, 0))
        pygame.display.flip()
        clock.tick(frames_per_second)
    pygame.quit()
    return None 



In [59]:
ToH_NoWrap(3, 5)

In [60]:
def ToH_BiColor(n, speed_factor):
    """
    Inputs: 
    - Number of Disks n 
    - The frame speed factor speed_factor
    Plays the animation of solving the Bicolor Variant of Tower of Hanoi.
    """
    # Basic initialization
    screen_size = (screen_width, screen_height)  = (1000, 600)
    white = (255,255,255)
    red = (255, 0, 0)
    green = (0, 255, 0)
    black = (0, 0, 0)
    frames_per_second = speed_factor
    clock = pygame.time.Clock()
    pygame.init()
    screen = pygame.display.set_mode(screen_size)
    # Title bar
    caption = 'Tower of Hanoi - Bicolor Variant'
    caption += '                              '
    caption += '(Keystroke:  \'Space\' to start or pause)'
    pygame.display.set_caption(caption)
    # Draw the background
    bg = pygame.image.load("ToH_BG.jpg")
    bg_rect = pygame.Rect(0,0,screen_width,screen_height)
    # Draw the disks
    get_double_disk_length = lambda d_n: 20 * d_n
    Sol_BiColor_Hanoi = ToH_Bicolor_Solve(n)
    len_move_list = len(Sol_BiColor_Hanoi)
    Initial_Configuration = [list(range(1, n+1, 2)), list(), list(range(2, n+1, 2))]
    Current_Configuration = Initial_Configuration
    Start_Coord_List = ToH_Get_Coordinate(Initial_Configuration)
    for x_cor, y_cor, disk in Start_Coord_List:
        disk_rect = pygame.Rect(x_cor, y_cor, get_double_disk_length(disk), 15)
        if disk % 2 ==0:
            pygame.draw.rect(bg, green, disk_rect)
        else:
            pygame.draw.rect(bg, red, disk_rect)
        pygame.draw.rect(bg, black, disk_rect, width = 3)
    # Initialization of animation
    # Index_move will track through the solution for the puzzle
    index_move = 0
    screen.fill(white)
    screen.blit(bg, bg_rect)
    pygame.display.flip()
    running = True
    run_anim = False
    # Main Loop 
    while running :
        # Key Press Detecting
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                run_anim = not run_anim 
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                frames_per_second = (4/3)* frames_per_second
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                frames_per_second = 0.75 * frames_per_second
            # Title bar
            caption = 'Tower of Hanoi - Classical Variant'
            caption += f' Speed = {int(frames_per_second)}             '
            caption += '(Keystroke:  \'Space\' to start or pause, Up/Down for speed adjusting))'
            pygame.display.set_caption(caption)
        # Stop animating when the end of move list is reached
        if run_anim and index_move < len_move_list:
            screen.fill(white)
            # Background has changed, update it again
            bg = pygame.image.load("ToH_BG.jpg")
            # Extract Current Configuration
            current_move = Sol_BiColor_Hanoi[index_move]
            Current_Configuration = ToH_Valid_Move(current_move,Current_Configuration)
            # Print Information
            my_font = pygame.font.SysFont('Comic Sans MS', 30)
            text_surface = my_font.render(f'Move {index_move+1}/{len_move_list}: {current_move}', False, (255, 0, 0))
            bg.blit(text_surface, (0, 0))
            # Update iterator
            index_move = (index_move + 1)
            ans = ToH_Get_Coordinate(Current_Configuration)
            # Draw the new disks
            for x_cor, y_cor, disk in ans:
                disk_rect = pygame.Rect(x_cor, y_cor, get_double_disk_length(disk), 15)
                if disk % 2 ==0:
                    pygame.draw.rect(bg, green, disk_rect)
                else:
                    pygame.draw.rect(bg, red, disk_rect)
                pygame.draw.rect(bg, black, disk_rect, width = 3)
        # Re-initialise the display 
        screen.blit(bg, (15, 0))
        pygame.display.flip()
        clock.tick(frames_per_second)
    pygame.quit()
    return None 

In [61]:
ToH_BiColor(4, 8)

From here, we can let the user choose the mode, discs, and speed factor using an interactive command.

In [62]:
def ToH_Animation_Interactive(): 
    Mode = input('Which variant? c - classical, n - nowrap, b - bicolor, q- quit')
    if Mode == 'c':
        ds = integer_input_validation("Number of disks", 5, 1, 16)
        spe = integer_input_validation("Disks moved per second", 5, 1, 20)
        ToH_Basic(ds, spe)
    elif Mode == 'n':
        ds = integer_input_validation("Number of disks", 6, 2, 16)
        spe = integer_input_validation("Disks moved per second", 5, 1, 20)
        ToH_NoWrap(ds, spe)
    elif Mode == 'b':
        ds = integer_input_validation("Number of disks", 5, 1, 16)
        spe = integer_input_validation("Disks moved per second", 5, 1, 20)
        ToH_BiColor(ds, spe)
    elif Mode == 'q':
        print('Quitting! Bye!')
    else:
        print("Cannot recognize your input! So I quit! Please run the function again.")


In [63]:
ToH_Animation_Interactive()

Quitting! Bye!


<a id='S5.5'></a> 
#### Part 2.2.5 Extensions

There are a lot of features we can add to the functions introduced in this part. Firstly, we look at implementing a smooth version of ToH_Basic, where the discs transfer smoothly.

In [64]:
def ToH_Basic_Smooth(n, speed_factor):
    """
    Inputs: 
    - Number of Disks n 
    - The frame speed factor speed_factor
    Return: None
    Plays the animation of solving the Classic Variant of Tower of Hanoi with animation of disks moving pole to pole.
    """
    x_step = 10
    y_step = 5
    # Basic initialization
    screen_size = (screen_width, screen_height)  = (1000, 600)
    white = (255,255,255)
    blue = (0, 0, 255)
    black = (0, 0, 0)
    frames_per_second = speed_factor
    clock = pygame.time.Clock()
    pygame.init()
    screen = pygame.display.set_mode(screen_size)
    
    # Draw the background
    bg = pygame.image.load("ToH_BG.jpg")
    bg_rect = pygame.Rect(0,0,screen_width,screen_height)
    # Draw the disks
    get_double_disk_length = lambda d_n: 20 * d_n
    Sol_Classic_Hanoi = ToH_Move_Left(n)
    len_move_list = len(Sol_Classic_Hanoi)
    Initial_Configuration = [list(range(1, n+1)), list(), list()]
    Current_Configuration = Initial_Configuration
    Start_Coord_List = ToH_Get_Coordinate(Initial_Configuration)
    ans = ToH_Get_Coordinate(Initial_Configuration)
    # Initialization of smooth disks
    current_disk = 1
    for x_cor, y_cor, disk in Start_Coord_List:
        disk_rect = pygame.Rect(x_cor, y_cor, get_double_disk_length(disk), 15)
        if disk == current_disk:
            moving_disk_rect = disk_rect
        pygame.draw.rect(bg, blue, disk_rect)
        pygame.draw.rect(bg, black, disk_rect, width = 3)
    # Initalization of the first move
    index_move = 0
    current_move = Sol_Classic_Hanoi[index_move]
    current_disk = current_move.disk
    current_direction = current_move.direction
    screen.fill(white)
    screen.blit(bg, bg_rect)
    pygame.display.flip()
    running = True
    run_anim = False
    smooth_disk = True
    number_of_times_moving_x = 0
    my_font = pygame.font.SysFont('Comic Sans MS', 30)
    text_surface = my_font.render(f'Move {index_move+1}: {current_move}', False, (255, 0, 0))
    bg.blit(text_surface, (0, 0))
    # Main Loop
    while running:
        # Key Press Detecting
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                run_anim = not run_anim 
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_UP:
                frames_per_second = (4/3)* frames_per_second
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_DOWN:
                frames_per_second = 0.75 * frames_per_second
            # Title bar
            caption = 'Tower of Hanoi - Classical Variant with Smooth Disks'
            caption += f' Speed = {int(frames_per_second)}             '
            caption += '(Keystroke:  \'Space\' to start/pause, Up/Down for speed adjusting)'
            pygame.display.set_caption(caption)
        if run_anim and index_move < len_move_list:
            # Smooth Transition Parts
            if smooth_disk:
                if number_of_times_moving_x < 25:
                    bg = pygame.image.load("ToH_BG.jpg")
                    move_hori = True
                    if current_direction == 'Left':
                        moving_disk_rect = moving_disk_rect.move((-x_step, 0))
                    else:
                        moving_disk_rect = moving_disk_rect.move((x_step, 0))
                    pygame.draw.rect(bg, blue ,moving_disk_rect)
                    pygame.draw.rect(bg, black ,moving_disk_rect, width = 3)
                    number_of_times_moving_x += 1
                else:
                    number_of_times_moving_x = 0
                    smooth_disk = not smooth_disk
                for x_cor, y_cor, disk in ans:
                    disk_rect = pygame.Rect(x_cor, y_cor, get_double_disk_length(disk), 15)
                    if disk != current_disk:
                        pygame.draw.rect(bg, blue, disk_rect)
                        pygame.draw.rect(bg, black, disk_rect, width = 3)
                my_font = pygame.font.SysFont('Comic Sans MS', 30)
                text_surface = my_font.render(f'Move {index_move+2}: {current_move}', False, (255, 0, 0))
                bg.blit(text_surface, (0, 0))
                clock.tick(frames_per_second)
                pygame.display.flip()
            elif index_move < len_move_list:
                screen.fill(white)
                # Background has changed, update it again
                bg = pygame.image.load("ToH_BG.jpg")
                # Extract Current Configuration
                try:
                    current_move = Sol_Classic_Hanoi[index_move+1]
                except:
                    pass
                current_disk = current_move.disk
                current_direction = current_move.direction
                Current_Configuration = ToH_Valid_Move(Sol_Classic_Hanoi[index_move],Current_Configuration)
                # Print Information
                my_font = pygame.font.SysFont('Comic Sans MS', 30)
                text_surface = my_font.render(f'Move {index_move+2}: {current_move}', False, (255, 0, 0))
                bg.blit(text_surface, (0, 0))
                # Update iterator
                ans = ToH_Get_Coordinate(Current_Configuration)
                # Draw the new disks
                for x_cor, y_cor, disk in ans:
                    disk_rect = pygame.Rect(x_cor, y_cor, get_double_disk_length(disk), 15)
                    if disk == current_disk:
                        moving_disk_rect = disk_rect
                    pygame.draw.rect(bg, blue, disk_rect)
                    pygame.draw.rect(bg, black, disk_rect, width = 3)
                smooth_disk = not smooth_disk
                index_move = (index_move + 1) 
        # Re-initialise the display 
        screen.blit(bg, (0, 0))
        pygame.display.flip()
        clock.tick(frames_per_second)
    pygame.quit()
    return None 

In [65]:
# Test Cells
ToH_Basic_Smooth(4, 60) 

Now as promised in the previous parts, we look at different time and space complexity issue. 

First, recall that we discussed the classical amnd no wrap variant have optimal length of solution $2^n-1$ and $3^n-1$ respectively. The informal reason of why they are optimal is as follows:

- In the classical variant, there must be one move where disk $n$ is moved from peg $A$ to $C$, before and after that, the move can be copied from the optimal moves in the solution of $n-1$ disks to solve the puzzle. That explains the $T(n-1)$ term in the recurrence relation.

- Similarly, in the no wrap variant, there must be two move that move disk $n$ from peg $A$ to $B$, then peg $B$ to $C$, that leaves three 'gaps' in between the two moves. That's why there are three $T(n-1)$ term present.

So in conclusion, the code for `ToH_Move_Left` and `ToH_NoWrap_Move_Left` is fully optimized in terms of solution length. However, we note that there exists numerous iterative solution that involve searching a valid move under a configuration (see the 'Solution' tab in [(Wikipedia Contributors, 2022)](#Wiki)). In our project, this may mean searching in a loop inside the function `ToH_Valid_Move`, which, to the authors, is an unncessary complication which may cause further runtime loss.

However, We shall look at how memoization speeds up the function call for large values of $n$ to show the importance of this technique.  We first start by implementing the no memoization version of functions that produces a solution, with a suffix `_NoMem` after their usual names in order to distinguish them.


In [66]:
def ToH_Move_Left_Nomem(n):
    """
    The function ToH_Move_Left with no memoization.
    """
    if n == 1:
        return [Move(1, 'Left')]
    else:
        MiddleMove = Move(n, 'Left')
        return ToH_Move_Right_Nomem(n-1) + [MiddleMove] + ToH_Move_Right_Nomem(n-1)
def ToH_Move_Right_Nomem(n):
    """
    The function ToH_Move_Right with no memoization.
    """
    if n == 1:
        return [Move(1, 'Right')]
    else:
        MiddleMove = Move(n, 'Right')
        return ToH_Move_Left_Nomem(n-1) + [MiddleMove] + ToH_Move_Left_Nomem(n-1)
def ToH_NoWrap_Move_Left_Nomem(n):
    """
    The function ToH_NoWrap_Move_Left with no memoization.
    """
    if n == 1:
        return [Move(1, 'Right'), Move(1, 'Right')]
    else:
        MiddleMove = Move(n, 'Right')
        return ToH_NoWrap_Move_Left_Nomem(n-1) + [MiddleMove] + \
    ToH_NoWrap_Move_Right_Nomem(n-1) + [MiddleMove] + \
    ToH_NoWrap_Move_Left_Nomem(n-1)
def ToH_NoWrap_Move_Right_Nomem(n):
    """
    The function ToH_NoWrap_Move_Right with no memoization.
    """
    if n == 1:
        return [Move(1, 'Left'), Move(1, 'Left')]
    else:
        MiddleMove = Move(n, 'Left')
        return ToH_NoWrap_Move_Right_Nomem(n-1) + [MiddleMove]  +\
    ToH_NoWrap_Move_Left_Nomem(n-1) + [MiddleMove] +  ToH_NoWrap_Move_Right_Nomem(n-1)
def ToH_To_Center_Nomem(n):
    """
    The function ToH_To_Center with no memoization.
    """
    if n == 1:
        return [Move(1, 'Right')]
    elif n ==2:
        return [Move(2, 'Left'), Move(1, 'Right')]
    elif n % 2 == 0: 
        MiddleMove = Move(n, 'Left')
        return ToH_To_Center_Nomem(n-2) + ToH_Move_Left_Nomem(n-2) + [MiddleMove] +ToH_Move_Right_Nomem(n-1)
    else: 
        MiddleMove = Move(n, 'Right')
        return ToH_To_Center_Nomem(n-1) + ToH_Move_Right_Nomem(n-1) + [MiddleMove] +ToH_Move_Left_Nomem(n-1)
def ToH_Bicolor_Solve_Nomem(n):
    """
    The function ToH_BiColor_Solve with no memoization.
    """
    even_error = 'n must be an even number!'
    if n % 2 != 0:
        print(even_error)
    elif n == 2:
        return [Move(2, 'Left'), Move(1, 'Left'), Move(2, 'Left')]
    else:
        MiddleMove = Move(n, 'Right')
        CenterMove = ToH_To_Center_Nomem(n-1)
        return CenterMove + [MiddleMove] + rev_list(CenterMove)

In [67]:
# Test Cell: The contemt of the no memoization version is the same as that of the original, memomized verion.
all([ToH_Move_Left_Nomem(i) == ToH_Move_Left(i) for i in range(1, 9)])

True

In [68]:
%timeit ToH_Move_Left(12)
%timeit ToH_Move_Left_Nomem(12)

289 ns ± 19.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
4.59 ms ± 891 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [69]:
all([ToH_NoWrap_Move_Left_Nomem(i) == ToH_NoWrap_Move_Left(i) for i in range(1, 9)])

True

In [70]:
%timeit ToH_NoWrap_Move_Left(12) 
%timeit ToH_NoWrap_Move_Left_Nomem(12) 

277 ns ± 13.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
524 ms ± 23.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
all([ToH_Bicolor_Solve_Nomem(i) == ToH_Bicolor_Solve(i) for i in range(2, 12, 2)])

True

In [72]:
%timeit ToH_Bicolor_Solve(12)
%timeit ToH_Bicolor_Solve_Nomem(12)

210 ns ± 5.56 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
3.51 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


As seen, as $n$ increases, the run time for no memoization dramatically increases, but with memoization, the running time can be kept to around $200$ nanoseconds.

<a id='Conc'></a>
## Conclusion 

Throughout the project, we have explained the complex topic of fractals and we have seen examples of how mathematical concepts like recursion can be implemented using various computational techniques. As Mandelbrot himself [(Benoît Mandelbrot, 1983)](#M1983) put it:

> *“Fractals Geometry reveals that some of the most austerely formal chapters of mathematics had a hidden face: a world of pure plastic beauty unsuspected till now.”*
>
> Benoît Mandelbrot, 1983

<a id='Bib'></a>
## References and Bibilography

<a id='Chen2015'></a> Chen, Y.L. (2015). *Efficient Algorithm for Tower of Hanoi Variation*. [online] Stack Overflow. Available at: https://stackoverflow.com/questions/32463594/efficient-algorithm-for-tower-of-hanoi-variation [Accessed 3 May 2022].

<a id='Falconer1990'></a> Falconer, K.J. (1990). *Fractal Geometry*. Wiley.

<a id='HKP2018'></a> Hinz, A.M., Klavzar, S. and Petr, C. (2018). *The Tower of Hanoi - Myths and Maths*. [online] Cham: Springer International Publishing. Available at: https://link.springer.com/content/pdf/10.1007/978-3-319-73779-9.pdf [Accessed 3 Apr. 2022].

<a id='Leung2021'></a> Leung, D. (2021). *Understanding Interfaces in Go*. [online] Duncanleung.com. Available at: https://duncanleung.com/understand-go-golang-interfaces/ [Accessed 17 Apr. 2022].

<a id='M1980'></a> Mandelbrot, B.B. (1980). FRACTAL ASPECTS OF THE ITERATION OF $z → \lambda z(1- z)$ FOR COMPLEX $\lambda$ AND $z$. *Annals of the New York Academy of Sciences*, [online] 357(1), pp.249–259. doi:10.1111/j.1749-6632.1980.tb29690.x.

<a id='M1983'></a> Mandelbrot, B.B. (1983). *The Fractal Geometry of Nature*. New York: W.H. Freeman and Company.

<a id='M1999'></a> Mandelbrot, B.B. and Frame, M. (1999). The Canopy and Shortest Path in a Self-Contacting Fractal Tree. *The Mathematical Intelligencer*, 21(2), pp.18–27. doi:10.1007/bf03024842.

<a id='PSF2022'></a> Python Software Foundation (2022). *9. Classes - Python 3.10.4 Documentation*. [online] Python.org. Available at: https://docs.python.org/3/tutorial/classes.html [Accessed 3 Apr. 2022].

<a id='RUP2008'></a> Rubio-Sánchez, M., Urquiza-Fuentes, J. and Pareja-Flores, C. (2008). A Gentle Introduction to Mutual Recursion. *Proceedings of the 13th Annual Conference on Innovation and Technology in Computer Science Education*, 2008(June). doi:10.1145/1384271.1384334.

<a id='PyGame_Intro'></a> Shinners, P. (2021). *Pygame Intro — Pygame v2.1.1 Documentation*. [online] Pygame.org. Available at: http://www.pygame.org/docs/tut/PygameIntro.html [Accessed 5 May 2022].

<a id='Wiki'></a> Wikipedia Contributors (2022). *Tower of Hanoi*. [online] Wikipedia. Available at: https://en.wikipedia.org/wiki/Tower_of_Hanoi#cite_ref-8 [Accessed 9 May 2022].
